In [1]:
import json
import os
import random
import subprocess

from anthropic import Anthropic
from dotenv import load_dotenv, find_dotenv
from IPython.display import display, Markdown
from openai import OpenAI
import tiktoken
from functions import *

load_dotenv(find_dotenv())

True

In [2]:
OpenAI_client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
Anthropic_client = Anthropic(api_key=os.environ.get("ANTHROPIC_API_KEY"),
)

In [3]:
file_path = 'instructions.txt'
instructions = read_file(file_path)

folder_path = '../foundry_hook_playground/src/examples'
file_to_code = read_all_files_in_folder(folder_path)

with open("hook_examples.json", 'r') as file:
    hook_examples_json = json.load(file)

In [4]:
len(file_to_code.keys())

15

In [5]:
rag_instructions = f"""
You have a JSON object mapping file names to summaries:

{hook_examples_json}

You will receive a user prompt and your task is to determine which Solidity files are most relevant to the prompt.

### Instructions:

1. **Analyze the user prompt**: Understand the key concepts and requirements from the user's prompt.
2. **Match with summaries**: Compare these key concepts with the summaries provided in the JSON object.
3. **Evaluate relevance**: Assign a confidence level to each file based on how well it matches the user's prompt. Use the following confidence levels:
   - High: Very relevant to the prompt
   - Medium: Somewhat relevant to the prompt
   - Low: Slightly relevant to the prompt

### Output Format:

Return a ONLY a JSON object with the top 5 most relevant files based on your evaluation. Each entry should have the following format:
file_name: high/medium/low

Return ONLY a JSON with 5 entries
"""

In [6]:
prompt = "I want a hook that does kyc so that if you dont have a certain nft you cant trade, and also it only allows to trade when the hour is pair"

rag_answer, conversation_history = get_openai_answer(rag_instructions, [], prompt, json_output=True)

In [7]:
eval(rag_answer)

{'KYCHook.sol': 'High',
 'ERC721OwnershipHook.sol': 'High',
 'TradingHours.sol': 'High',
 'WorldIDHook.sol': 'Medium',
 'WhiteListHook.sol': 'Low'}

In [8]:
final_instructions = instructions
counter = 1

for file in eval(rag_answer).keys():
    if file not in list(hook_examples_json.keys()):
        print(f"File {file} does not exist!")
    else:
        final_instructions += f"""----------\nHOOK EXAMPLE {counter}:\n\n
        SUMMARY: {hook_examples_json[file]}\n
        CODE:\n {file_to_code[file]}\n\n\n------------------\n\n\n"""
        counter+=1

final_instructions+="OUTPUT: ONLY Solidity code, nothing else - no explanations, summaries or descriptions. ONLY working Solidity code, WITH comments."

In [9]:
get_n_tokens(final_instructions)

6774

In [10]:
attempt_counter = 0
conversation_history = []
returncode =- 1

while (attempt_counter<5) and (returncode!=0):
    file_name = f"test_{attempt_counter}.sol"
    #answer, conversation_history = get_claude_answer(final_instructions, conversation_history, prompt)
    answer, conversation_history = get_openai_answer(final_instructions, conversation_history, prompt)
    answer = markdown_to_text(answer)
    save_to_sol(answer, "../foundry_hook_playground/src/generated/", file_name)
    stdout, stderr, returncode = compile_contract(file_name)
    prompt = "I get this error when compiling the contract: \n\n"+stderr+"\n\nOUTPUT: Only the fixed solidity code"
    if (returncode)!=0:
        print(stderr)
        print("-----------------------")
    else:
        print("\U0001F389 \U0001F389 Contract compiled! \U0001F389 \U0001F389")
    attempt_counter+=1

🎉 🎉 Contract compiled! 🎉 🎉


In [11]:
true=True
false=False
a = {"abi":[{"type":"constructor","inputs":[{"name":"_poolManager","type":"address","internalType":"contract IPoolManager"}],"stateMutability":"nonpayable"},{"type":"function","name":"afterAddLiquidity","inputs":[{"name":"","type":"address","internalType":"address"},{"name":"","type":"tuple","internalType":"struct PoolKey","components":[{"name":"currency0","type":"address","internalType":"Currency"},{"name":"currency1","type":"address","internalType":"Currency"},{"name":"fee","type":"uint24","internalType":"uint24"},{"name":"tickSpacing","type":"int24","internalType":"int24"},{"name":"hooks","type":"address","internalType":"contract IHooks"}]},{"name":"","type":"tuple","internalType":"struct IPoolManager.ModifyLiquidityParams","components":[{"name":"tickLower","type":"int24","internalType":"int24"},{"name":"tickUpper","type":"int24","internalType":"int24"},{"name":"liquidityDelta","type":"int256","internalType":"int256"},{"name":"salt","type":"bytes32","internalType":"bytes32"}]},{"name":"","type":"int256","internalType":"BalanceDelta"},{"name":"","type":"bytes","internalType":"bytes"}],"outputs":[{"name":"","type":"bytes4","internalType":"bytes4"},{"name":"","type":"int256","internalType":"BalanceDelta"}],"stateMutability":"nonpayable"},{"type":"function","name":"afterDonate","inputs":[{"name":"","type":"address","internalType":"address"},{"name":"","type":"tuple","internalType":"struct PoolKey","components":[{"name":"currency0","type":"address","internalType":"Currency"},{"name":"currency1","type":"address","internalType":"Currency"},{"name":"fee","type":"uint24","internalType":"uint24"},{"name":"tickSpacing","type":"int24","internalType":"int24"},{"name":"hooks","type":"address","internalType":"contract IHooks"}]},{"name":"","type":"uint256","internalType":"uint256"},{"name":"","type":"uint256","internalType":"uint256"},{"name":"","type":"bytes","internalType":"bytes"}],"outputs":[{"name":"","type":"bytes4","internalType":"bytes4"}],"stateMutability":"nonpayable"},{"type":"function","name":"afterInitialize","inputs":[{"name":"","type":"address","internalType":"address"},{"name":"","type":"tuple","internalType":"struct PoolKey","components":[{"name":"currency0","type":"address","internalType":"Currency"},{"name":"currency1","type":"address","internalType":"Currency"},{"name":"fee","type":"uint24","internalType":"uint24"},{"name":"tickSpacing","type":"int24","internalType":"int24"},{"name":"hooks","type":"address","internalType":"contract IHooks"}]},{"name":"","type":"uint160","internalType":"uint160"},{"name":"","type":"int24","internalType":"int24"},{"name":"","type":"bytes","internalType":"bytes"}],"outputs":[{"name":"","type":"bytes4","internalType":"bytes4"}],"stateMutability":"nonpayable"},{"type":"function","name":"afterRemoveLiquidity","inputs":[{"name":"","type":"address","internalType":"address"},{"name":"","type":"tuple","internalType":"struct PoolKey","components":[{"name":"currency0","type":"address","internalType":"Currency"},{"name":"currency1","type":"address","internalType":"Currency"},{"name":"fee","type":"uint24","internalType":"uint24"},{"name":"tickSpacing","type":"int24","internalType":"int24"},{"name":"hooks","type":"address","internalType":"contract IHooks"}]},{"name":"","type":"tuple","internalType":"struct IPoolManager.ModifyLiquidityParams","components":[{"name":"tickLower","type":"int24","internalType":"int24"},{"name":"tickUpper","type":"int24","internalType":"int24"},{"name":"liquidityDelta","type":"int256","internalType":"int256"},{"name":"salt","type":"bytes32","internalType":"bytes32"}]},{"name":"","type":"int256","internalType":"BalanceDelta"},{"name":"","type":"bytes","internalType":"bytes"}],"outputs":[{"name":"","type":"bytes4","internalType":"bytes4"},{"name":"","type":"int256","internalType":"BalanceDelta"}],"stateMutability":"nonpayable"},{"type":"function","name":"afterSwap","inputs":[{"name":"","type":"address","internalType":"address"},{"name":"","type":"tuple","internalType":"struct PoolKey","components":[{"name":"currency0","type":"address","internalType":"Currency"},{"name":"currency1","type":"address","internalType":"Currency"},{"name":"fee","type":"uint24","internalType":"uint24"},{"name":"tickSpacing","type":"int24","internalType":"int24"},{"name":"hooks","type":"address","internalType":"contract IHooks"}]},{"name":"","type":"tuple","internalType":"struct IPoolManager.SwapParams","components":[{"name":"zeroForOne","type":"bool","internalType":"bool"},{"name":"amountSpecified","type":"int256","internalType":"int256"},{"name":"sqrtPriceLimitX96","type":"uint160","internalType":"uint160"}]},{"name":"","type":"int256","internalType":"BalanceDelta"},{"name":"","type":"bytes","internalType":"bytes"}],"outputs":[{"name":"","type":"bytes4","internalType":"bytes4"},{"name":"","type":"int128","internalType":"int128"}],"stateMutability":"nonpayable"},{"type":"function","name":"beforeAddLiquidity","inputs":[{"name":"","type":"address","internalType":"address"},{"name":"","type":"tuple","internalType":"struct PoolKey","components":[{"name":"currency0","type":"address","internalType":"Currency"},{"name":"currency1","type":"address","internalType":"Currency"},{"name":"fee","type":"uint24","internalType":"uint24"},{"name":"tickSpacing","type":"int24","internalType":"int24"},{"name":"hooks","type":"address","internalType":"contract IHooks"}]},{"name":"","type":"tuple","internalType":"struct IPoolManager.ModifyLiquidityParams","components":[{"name":"tickLower","type":"int24","internalType":"int24"},{"name":"tickUpper","type":"int24","internalType":"int24"},{"name":"liquidityDelta","type":"int256","internalType":"int256"},{"name":"salt","type":"bytes32","internalType":"bytes32"}]},{"name":"","type":"bytes","internalType":"bytes"}],"outputs":[{"name":"","type":"bytes4","internalType":"bytes4"}],"stateMutability":"nonpayable"},{"type":"function","name":"beforeDonate","inputs":[{"name":"","type":"address","internalType":"address"},{"name":"","type":"tuple","internalType":"struct PoolKey","components":[{"name":"currency0","type":"address","internalType":"Currency"},{"name":"currency1","type":"address","internalType":"Currency"},{"name":"fee","type":"uint24","internalType":"uint24"},{"name":"tickSpacing","type":"int24","internalType":"int24"},{"name":"hooks","type":"address","internalType":"contract IHooks"}]},{"name":"","type":"uint256","internalType":"uint256"},{"name":"","type":"uint256","internalType":"uint256"},{"name":"","type":"bytes","internalType":"bytes"}],"outputs":[{"name":"","type":"bytes4","internalType":"bytes4"}],"stateMutability":"nonpayable"},{"type":"function","name":"beforeInitialize","inputs":[{"name":"","type":"address","internalType":"address"},{"name":"","type":"tuple","internalType":"struct PoolKey","components":[{"name":"currency0","type":"address","internalType":"Currency"},{"name":"currency1","type":"address","internalType":"Currency"},{"name":"fee","type":"uint24","internalType":"uint24"},{"name":"tickSpacing","type":"int24","internalType":"int24"},{"name":"hooks","type":"address","internalType":"contract IHooks"}]},{"name":"","type":"uint160","internalType":"uint160"},{"name":"","type":"bytes","internalType":"bytes"}],"outputs":[{"name":"","type":"bytes4","internalType":"bytes4"}],"stateMutability":"nonpayable"},{"type":"function","name":"beforeRemoveLiquidity","inputs":[{"name":"","type":"address","internalType":"address"},{"name":"","type":"tuple","internalType":"struct PoolKey","components":[{"name":"currency0","type":"address","internalType":"Currency"},{"name":"currency1","type":"address","internalType":"Currency"},{"name":"fee","type":"uint24","internalType":"uint24"},{"name":"tickSpacing","type":"int24","internalType":"int24"},{"name":"hooks","type":"address","internalType":"contract IHooks"}]},{"name":"","type":"tuple","internalType":"struct IPoolManager.ModifyLiquidityParams","components":[{"name":"tickLower","type":"int24","internalType":"int24"},{"name":"tickUpper","type":"int24","internalType":"int24"},{"name":"liquidityDelta","type":"int256","internalType":"int256"},{"name":"salt","type":"bytes32","internalType":"bytes32"}]},{"name":"","type":"bytes","internalType":"bytes"}],"outputs":[{"name":"","type":"bytes4","internalType":"bytes4"}],"stateMutability":"nonpayable"},{"type":"function","name":"beforeSwap","inputs":[{"name":"","type":"address","internalType":"address"},{"name":"key","type":"tuple","internalType":"struct PoolKey","components":[{"name":"currency0","type":"address","internalType":"Currency"},{"name":"currency1","type":"address","internalType":"Currency"},{"name":"fee","type":"uint24","internalType":"uint24"},{"name":"tickSpacing","type":"int24","internalType":"int24"},{"name":"hooks","type":"address","internalType":"contract IHooks"}]},{"name":"","type":"tuple","internalType":"struct IPoolManager.SwapParams","components":[{"name":"zeroForOne","type":"bool","internalType":"bool"},{"name":"amountSpecified","type":"int256","internalType":"int256"},{"name":"sqrtPriceLimitX96","type":"uint160","internalType":"uint160"}]},{"name":"","type":"bytes","internalType":"bytes"}],"outputs":[{"name":"","type":"bytes4","internalType":"bytes4"},{"name":"","type":"int256","internalType":"BeforeSwapDelta"},{"name":"","type":"uint24","internalType":"uint24"}],"stateMutability":"nonpayable"},{"type":"function","name":"getHookPermissions","inputs":[],"outputs":[{"name":"","type":"tuple","internalType":"struct Hooks.Permissions","components":[{"name":"beforeInitialize","type":"bool","internalType":"bool"},{"name":"afterInitialize","type":"bool","internalType":"bool"},{"name":"beforeAddLiquidity","type":"bool","internalType":"bool"},{"name":"afterAddLiquidity","type":"bool","internalType":"bool"},{"name":"beforeRemoveLiquidity","type":"bool","internalType":"bool"},{"name":"afterRemoveLiquidity","type":"bool","internalType":"bool"},{"name":"beforeSwap","type":"bool","internalType":"bool"},{"name":"afterSwap","type":"bool","internalType":"bool"},{"name":"beforeDonate","type":"bool","internalType":"bool"},{"name":"afterDonate","type":"bool","internalType":"bool"},{"name":"beforeSwapReturnDelta","type":"bool","internalType":"bool"},{"name":"afterSwapReturnDelta","type":"bool","internalType":"bool"},{"name":"afterAddLiquidityReturnDelta","type":"bool","internalType":"bool"},{"name":"afterRemoveLiquidityReturnDelta","type":"bool","internalType":"bool"}]}],"stateMutability":"pure"},{"type":"function","name":"manager","inputs":[],"outputs":[{"name":"","type":"address","internalType":"contract IPoolManager"}],"stateMutability":"view"},{"type":"function","name":"swapCount","inputs":[{"name":"","type":"bytes32","internalType":"PoolId"}],"outputs":[{"name":"","type":"uint256","internalType":"uint256"}],"stateMutability":"view"},{"type":"function","name":"unlockCallback","inputs":[{"name":"data","type":"bytes","internalType":"bytes"}],"outputs":[{"name":"","type":"bytes","internalType":"bytes"}],"stateMutability":"nonpayable"},{"type":"error","name":"HookNotImplemented","inputs":[]},{"type":"error","name":"InvalidPool","inputs":[]},{"type":"error","name":"LockFailure","inputs":[]},{"type":"error","name":"NotManager","inputs":[]},{"type":"error","name":"NotSelf","inputs":[]}],"bytecode":{"object":"0x60a060405234801561000f575f80fd5b50604051610de5380380610de583398101604081905261002e9161029b565b6001600160a01b038116608052806100453061004c565b50506102c8565b61013c81610137604080516101c0810182525f80825260208201819052918101829052606081018290526080810182905260a0810182905260c0810182905260e08101829052610100810182905261012081018290526101408101829052610160810182905261018081018290526101a081019190915250604080516101c0810182525f80825260208201819052918101829052606081018290526080810182905260a08101829052600160c082015260e08101829052610100810182905261012081018290526101408101829052610160810182905261018081018290526101a081019190915290565b61013f565b50565b80511515612000831615151415806101635750602081015115156110008316151514155b8061017a5750604081015115156108008316151514155b806101915750606081015115156104008316151514155b806101a85750608081015115156102008316151514155b806101bf575060a081015115156101008316151514155b806101d5575060c0810151151560808316151514155b806101eb575060e0810151151560408316151514155b806102025750610100810151151560208316151514155b806102195750610120810151151560108316151514155b806102305750610140810151151560088316151514155b806102475750610160810151151560048316151514155b8061025e5750610180810151151560028316151514155b8061027557506101a0810151151560018316151514155b1561028b5761028b630732d7b560e51b8361028f565b5050565b815f528060045260245ffd5b5f602082840312156102ab575f80fd5b81516001600160a01b03811681146102c1575f80fd5b9392505050565b608051610afe6102e75f395f818161012d01526104000152610afe5ff3fe608060405234801561000f575f80fd5b50600436106100e5575f3560e01c806391dd734611610088578063b47b2fb111610063578063b47b2fb114610233578063b6a8b0fa14610264578063c4e833ce14610272578063e1b4af6914610264575f80fd5b806391dd7346146101d857806394168af8146101f8578063a910f80f14610225575f80fd5b8063481c6a75116100c3578063481c6a7514610128578063575e24b4146101675780635a2a8100146101a55780638db2b652146101a5575f80fd5b806321d0ee70146100e9578063259982e5146100e95780633440d8201461011a575b5f80fd5b6100fc6100f736600461057c565b610366565b6040516001600160e01b031990911681526020015b60405180910390f35b6100fc6100f73660046105f3565b61014f7f000000000000000000000000000000000000000000000000000000000000000081565b6040516001600160a01b039091168152602001610111565b61017a61017536600461065d565b610380565b604080516001600160e01b03199094168452602084019290925262ffffff1690820152606001610111565b6101b86101b33660046106b7565b6103d8565b604080516001600160e01b03199093168352602083019190915201610111565b6101eb6101e6366004610738565b6103f3565b6040516101119190610777565b6102176102063660046107ac565b5f6020819052908152604090205481565b604051908152602001610111565b6100fc6100f73660046107d4565b6102416101b336600461083e565b604080516001600160e01b03199093168352600f9190910b602083015201610111565b6100fc6100f73660046108a1565b610359604080516101c0810182525f80825260208201819052918101829052606081018290526080810182905260a0810182905260c0810182905260e08101829052610100810182905261012081018290526101408101829052610160810182905261018081018290526101a081019190915250604080516101c0810182525f80825260208201819052918101829052606081018290526080810182905260a08101829052600160c082015260e08101829052610100810182905261012081018290526101408101829052610160810182905261018081018290526101a081019190915290565b60405161011191906108fb565b5f604051630a85dc2960e01b815260040160405180910390fd5b5f8080808061039e610397368b90038b018b6109f9565b60a0902090565b81526020019081526020015f205f8154809291906103bb90610a95565b909155506315d7892d60e21b995f99508998509650505050505050565b5f80604051630a85dc2960e01b815260040160405180910390fd5b6060336001600160a01b037f0000000000000000000000000000000000000000000000000000000000000000161461043e5760405163607e454560e11b815260040160405180910390fd5b6104488383610451565b90505b92915050565b60605f80306001600160a01b0316858560405161046f929190610ab9565b5f604051808303815f865af19150503d805f81146104a8576040519150601f19603f3d011682016040523d82523d5f602084013e6104ad565b606091505b509150915081156104c157915061044b9050565b80515f036104e2576040516314815f4760e31b815260040160405180910390fd5b805160208201fd5b6001600160a01b03811681146104fe575f80fd5b50565b803561050c816104ea565b919050565b5f60a08284031215610521575f80fd5b50919050565b5f60808284031215610521575f80fd5b5f8083601f840112610547575f80fd5b50813567ffffffffffffffff81111561055e575f80fd5b602083019150836020828501011115610575575f80fd5b9250929050565b5f805f805f6101608688031215610591575f80fd5b853561059c816104ea565b94506105ab8760208801610511565b93506105ba8760c08801610527565b925061014086013567ffffffffffffffff8111156105d6575f80fd5b6105e288828901610537565b969995985093965092949392505050565b5f805f805f6101008688031215610608575f80fd5b8535610613816104ea565b94506106228760208801610511565b935060c0860135610632816104ea565b925060e086013567ffffffffffffffff8111156105d6575f80fd5b5f60608284031215610521575f80fd5b5f805f805f6101408688031215610672575f80fd5b853561067d816104ea565b945061068c8760208801610511565b935061069b8760c0880161064d565b925061012086013567ffffffffffffffff8111156105d6575f80fd5b5f805f805f8061018087890312156106cd575f80fd5b86356106d8816104ea565b95506106e78860208901610511565b94506106f68860c08901610527565b9350610140870135925061016087013567ffffffffffffffff81111561071a575f80fd5b61072689828a01610537565b979a9699509497509295939492505050565b5f8060208385031215610749575f80fd5b823567ffffffffffffffff81111561075f575f80fd5b61076b85828601610537565b90969095509350505050565b602081525f82518060208401528060208501604085015e5f604082850101526040601f19601f83011684010191505092915050565b5f602082840312156107bc575f80fd5b5035919050565b8035600281900b811461050c575f80fd5b5f805f805f8061012087890312156107ea575f80fd5b86356107f5816104ea565b95506108048860208901610511565b945060c0870135610814816104ea565b935061082260e088016107c3565b925061010087013567ffffffffffffffff81111561071a575f80fd5b5f805f805f806101608789031215610854575f80fd5b863561085f816104ea565b955061086e8860208901610511565b945061087d8860c0890161064d565b9350610120870135925061014087013567ffffffffffffffff81111561071a575f80fd5b5f805f805f8061012087890312156108b7575f80fd5b86356108c2816104ea565b95506108d18860208901610511565b945060c0870135935060e0870135925061010087013567ffffffffffffffff81111561071a575f80fd5b8151151581526101c081016020830151610919602084018215159052565b50604083015161092d604084018215159052565b506060830151610941606084018215159052565b506080830151610955608084018215159052565b5060a083015161096960a084018215159052565b5060c083015161097d60c084018215159052565b5060e083015161099160e084018215159052565b5061010083810151151590830152610120808401511515908301526101408084015115159083015261016080840151151590830152610180808401511515908301526101a0928301511515929091019190915290565b803562ffffff8116811461050c575f80fd5b5f60a08284031215610a09575f80fd5b60405160a0810181811067ffffffffffffffff82111715610a3857634e487b7160e01b5f52604160045260245ffd5b6040528235610a46816104ea565b81526020830135610a56816104ea565b6020820152610a67604084016109e7565b6040820152610a78606084016107c3565b6060820152610a8960808401610501565b60808201529392505050565b5f60018201610ab257634e487b7160e01b5f52601160045260245ffd5b5060010190565b818382375f910190815291905056fea2646970667358221220ef3bcb492dfdfe4dc20155d5dafd2cd7418d31f6bbced538aecdd711ce7d02ab64736f6c63430008190033","sourceMap":"519:1449:88:-:0;;;750:64;;;;;;;;;;;;;;;;;;;;;;;;;;;;:::i;:::-;-1:-1:-1;;;;;270:18:3;;;;798:12:88;837:25:2::1;857:4;837:19;:25::i;:::-;767:102:::0;750:64:88;519:1449;;1513:142:2;1590:58;1620:5;1627:20;519:1449:88;;;;;;;;-1:-1:-1;519:1449:88;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;-1:-1:-1;1006:571:88;;;;;;;;-1:-1:-1;1006:571:88;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;903:681;1627:20:2;1590:29;:58::i;:::-;1513:142;:::o;3495:1599:48:-;3614:28;;:74;;1321:7;16076:29;;:34;;3614:74;;;:166;;-1:-1:-1;3708:27:48;;;;:72;;1384:7;16076:29;;:34;;3708:72;;3614:166;:265;;;-1:-1:-1;3800:30:48;;;;:79;;1452:7;16076:29;;:34;;3800:79;;3614:265;:362;;;-1:-1:-1;3899:29:48;;;;:77;;1518:7;16076:29;;:34;;3899:77;;3614:362;:467;;;-1:-1:-1;3996:33:48;;;;:85;;1589:6;16076:29;;:34;;3996:85;;3614:467;:570;;;-1:-1:-1;4101:32:48;;;;:83;;1657:6;16076:29;;:34;;4101:83;;3614:570;:652;;;-1:-1:-1;4204:22:48;;;;:62;;1715:6;16076:29;;:34;;4204:62;;3614:652;:732;;;-1:-1:-1;4286:21:48;;;;:60;;1771:6;16076:29;;:34;;4286:60;;3614:732;:818;;;-1:-1:-1;4366:24:48;;;;:66;;1831:6;16076:29;;:34;;4366:66;;3614:818;:902;;;-1:-1:-1;4452:23:48;;;;:64;;1889:6;16076:29;;:34;;4452:64;;3614:902;:1009;;;-1:-1:-1;4536:33:48;;;;:87;;1961:6;16076:29;;:34;;4536:87;;3614:1009;:1114;;;-1:-1:-1;4643:32:48;;;;:85;;2031:6;16076:29;;:34;;4643:85;;3614:1114;:1236;;;-1:-1:-1;4748:40:48;;;;:102;;2110:6;16076:29;;:34;;4748:102;;3614:1236;:1384;;;-1:-1:-1;4870:43:48;;;;:128;;2192:6;16076:29;;:34;;4870:128;;3614:1384;3597:1491;;;5023:54;-1:-1:-1;;;5071:4:48;5023:39;:54::i;:::-;3495:1599;;:::o;890:235:44:-;1041:8;1038:1;1031:19;1076:4;1070;1063:18;1104:4;1101:1;1094:15;14:312:89;106:6;159:2;147:9;138:7;134:23;130:32;127:52;;;175:1;172;165:12;127:52;201:16;;-1:-1:-1;;;;;246:31:89;;236:42;;226:70;;292:1;289;282:12;226:70;315:5;14:312;-1:-1:-1;;;14:312:89:o;:::-;519:1449:88;;;;;;;;;;;;;;;;;","linkReferences":{}},"deployedBytecode":{"object":"0x608060405234801561000f575f80fd5b50600436106100e5575f3560e01c806391dd734611610088578063b47b2fb111610063578063b47b2fb114610233578063b6a8b0fa14610264578063c4e833ce14610272578063e1b4af6914610264575f80fd5b806391dd7346146101d857806394168af8146101f8578063a910f80f14610225575f80fd5b8063481c6a75116100c3578063481c6a7514610128578063575e24b4146101675780635a2a8100146101a55780638db2b652146101a5575f80fd5b806321d0ee70146100e9578063259982e5146100e95780633440d8201461011a575b5f80fd5b6100fc6100f736600461057c565b610366565b6040516001600160e01b031990911681526020015b60405180910390f35b6100fc6100f73660046105f3565b61014f7f000000000000000000000000000000000000000000000000000000000000000081565b6040516001600160a01b039091168152602001610111565b61017a61017536600461065d565b610380565b604080516001600160e01b03199094168452602084019290925262ffffff1690820152606001610111565b6101b86101b33660046106b7565b6103d8565b604080516001600160e01b03199093168352602083019190915201610111565b6101eb6101e6366004610738565b6103f3565b6040516101119190610777565b6102176102063660046107ac565b5f6020819052908152604090205481565b604051908152602001610111565b6100fc6100f73660046107d4565b6102416101b336600461083e565b604080516001600160e01b03199093168352600f9190910b602083015201610111565b6100fc6100f73660046108a1565b610359604080516101c0810182525f80825260208201819052918101829052606081018290526080810182905260a0810182905260c0810182905260e08101829052610100810182905261012081018290526101408101829052610160810182905261018081018290526101a081019190915250604080516101c0810182525f80825260208201819052918101829052606081018290526080810182905260a08101829052600160c082015260e08101829052610100810182905261012081018290526101408101829052610160810182905261018081018290526101a081019190915290565b60405161011191906108fb565b5f604051630a85dc2960e01b815260040160405180910390fd5b5f8080808061039e610397368b90038b018b6109f9565b60a0902090565b81526020019081526020015f205f8154809291906103bb90610a95565b909155506315d7892d60e21b995f99508998509650505050505050565b5f80604051630a85dc2960e01b815260040160405180910390fd5b6060336001600160a01b037f0000000000000000000000000000000000000000000000000000000000000000161461043e5760405163607e454560e11b815260040160405180910390fd5b6104488383610451565b90505b92915050565b60605f80306001600160a01b0316858560405161046f929190610ab9565b5f604051808303815f865af19150503d805f81146104a8576040519150601f19603f3d011682016040523d82523d5f602084013e6104ad565b606091505b509150915081156104c157915061044b9050565b80515f036104e2576040516314815f4760e31b815260040160405180910390fd5b805160208201fd5b6001600160a01b03811681146104fe575f80fd5b50565b803561050c816104ea565b919050565b5f60a08284031215610521575f80fd5b50919050565b5f60808284031215610521575f80fd5b5f8083601f840112610547575f80fd5b50813567ffffffffffffffff81111561055e575f80fd5b602083019150836020828501011115610575575f80fd5b9250929050565b5f805f805f6101608688031215610591575f80fd5b853561059c816104ea565b94506105ab8760208801610511565b93506105ba8760c08801610527565b925061014086013567ffffffffffffffff8111156105d6575f80fd5b6105e288828901610537565b969995985093965092949392505050565b5f805f805f6101008688031215610608575f80fd5b8535610613816104ea565b94506106228760208801610511565b935060c0860135610632816104ea565b925060e086013567ffffffffffffffff8111156105d6575f80fd5b5f60608284031215610521575f80fd5b5f805f805f6101408688031215610672575f80fd5b853561067d816104ea565b945061068c8760208801610511565b935061069b8760c0880161064d565b925061012086013567ffffffffffffffff8111156105d6575f80fd5b5f805f805f8061018087890312156106cd575f80fd5b86356106d8816104ea565b95506106e78860208901610511565b94506106f68860c08901610527565b9350610140870135925061016087013567ffffffffffffffff81111561071a575f80fd5b61072689828a01610537565b979a9699509497509295939492505050565b5f8060208385031215610749575f80fd5b823567ffffffffffffffff81111561075f575f80fd5b61076b85828601610537565b90969095509350505050565b602081525f82518060208401528060208501604085015e5f604082850101526040601f19601f83011684010191505092915050565b5f602082840312156107bc575f80fd5b5035919050565b8035600281900b811461050c575f80fd5b5f805f805f8061012087890312156107ea575f80fd5b86356107f5816104ea565b95506108048860208901610511565b945060c0870135610814816104ea565b935061082260e088016107c3565b925061010087013567ffffffffffffffff81111561071a575f80fd5b5f805f805f806101608789031215610854575f80fd5b863561085f816104ea565b955061086e8860208901610511565b945061087d8860c0890161064d565b9350610120870135925061014087013567ffffffffffffffff81111561071a575f80fd5b5f805f805f8061012087890312156108b7575f80fd5b86356108c2816104ea565b95506108d18860208901610511565b945060c0870135935060e0870135925061010087013567ffffffffffffffff81111561071a575f80fd5b8151151581526101c081016020830151610919602084018215159052565b50604083015161092d604084018215159052565b506060830151610941606084018215159052565b506080830151610955608084018215159052565b5060a083015161096960a084018215159052565b5060c083015161097d60c084018215159052565b5060e083015161099160e084018215159052565b5061010083810151151590830152610120808401511515908301526101408084015115159083015261016080840151151590830152610180808401511515908301526101a0928301511515929091019190915290565b803562ffffff8116811461050c575f80fd5b5f60a08284031215610a09575f80fd5b60405160a0810181811067ffffffffffffffff82111715610a3857634e487b7160e01b5f52604160045260245ffd5b6040528235610a46816104ea565b81526020830135610a56816104ea565b6020820152610a67604084016109e7565b6040820152610a78606084016107c3565b6060820152610a8960808401610501565b60808201529392505050565b5f60018201610ab257634e487b7160e01b5f52601160045260245ffd5b5060010190565b818382375f910190815291905056fea2646970667358221220ef3bcb492dfdfe4dc20155d5dafd2cd7418d31f6bbced538aecdd711ce7d02ab64736f6c63430008190033","sourceMap":"519:1449:88:-:0;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;2712:235:2;;;;;;:::i;:::-;;:::i;:::-;;;-1:-1:-1;;;;;;1996:33:89;;;1978:52;;1966:2;1951:18;2712:235:2;;;;;;;;2126:156;;;;;;:::i;181:37:3:-;;;;;;;;-1:-1:-1;;;;;3047:55:89;;;3029:74;;3017:2;3002:18;181:37:3;2861:248:89;1642:324:88;;;;;;:::i;:::-;;:::i;:::-;;;;-1:-1:-1;;;;;;4341:33:89;;;4323:52;;4406:2;4391:18;;4384:34;;;;4466:8;4454:21;4434:18;;;4427:49;4311:2;4296:18;1642:324:88;4089:393:89;2953:267:2;;;;;;:::i;:::-;;:::i;:::-;;;;-1:-1:-1;;;;;;5647:33:89;;;5629:52;;5712:2;5697:18;;5690:34;;;;5602:18;2953:267:2;5424:306:89;614:136:4;;;;;;:::i;:::-;;:::i;:::-;;;;;;;:::i;638:43:88:-;;;;;;:::i;:::-;;;;;;;;;;;;;;;;;;6929:25:89;;;6917:2;6902:18;638:43:88;6783:177:89;2288:190:2;;;;;;:::i;3736:224::-;;;;;;:::i;:::-;;;;-1:-1:-1;;;;;;9124:33:89;;;9106:52;;9205:2;9194:22;;;;9189:2;9174:18;;9167:50;9079:18;3736:224:2;8936:287:89;3966:189:2;;;;;;:::i;903:681:88:-;;-1:-1:-1;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;1006:571:88;;;;;;;;-1:-1:-1;1006:571:88;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;903:681;;;;;;;;:::i;2712:235:2:-;2895:6;2920:20;;-1:-1:-1;;;2920:20:2;;;;;;;;;;;1642:324:88;1808:6;;;;;1861:10;:8;;;;;;;:3;:8;:::i;:::-;453:10:68;434:30;;;272:208;1861:10:88;1851:21;;;;;;;;;;;;:23;;;;;;;;;:::i;:::-;;;;-1:-1:-1;;;;1892:28:88;766:1:66;;-1:-1:-1;766:1:66;;-1:-1:-1;1642:324:88;-1:-1:-1;;;;;;;1642:324:88:o;2953:267:2:-;3154:6;3162:12;3193:20;;-1:-1:-1;;;3193:20:2;;;;;;;;;;;614:136:4;691:12;428:10;-1:-1:-1;;;;;450:7:4;428:30;;424:55;;467:12;;-1:-1:-1;;;467:12:4;;;;;;;;;;;424:55;722:21:::1;738:4;;722:15;:21::i;:::-;715:28;;489:1;614:136:::0;;;;:::o;1661:459:2:-;1742:12;1767;1781:23;1816:4;-1:-1:-1;;;;;1808:18:2;1827:4;;1808:24;;;;;;;:::i;:::-;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;1766:66;;;;1846:7;1842:30;;;1862:10;-1:-1:-1;1855:17:2;;-1:-1:-1;1855:17:2;1842:30;1886:10;:17;1907:1;1886:22;1882:48;;1917:13;;-1:-1:-1;;;1917:13:2;;;;;;;;;;;1882:48;2092:10;2086:17;2081:2;2069:10;2065:19;2058:46;14:154:89;-1:-1:-1;;;;;93:5:89;89:54;82:5;79:65;69:93;;158:1;155;148:12;69:93;14:154;:::o;173:134::-;241:20;;270:31;241:20;270:31;:::i;:::-;173:134;;;:::o;312:156::-;372:5;417:3;408:6;403:3;399:16;395:26;392:46;;;434:1;431;424:12;392:46;-1:-1:-1;456:6:89;312:156;-1:-1:-1;312:156:89:o;473:170::-;547:5;592:3;583:6;578:3;574:16;570:26;567:46;;;609:1;606;599:12;648:347;699:8;709:6;763:3;756:4;748:6;744:17;740:27;730:55;;781:1;778;771:12;730:55;-1:-1:-1;804:20:89;;847:18;836:30;;833:50;;;879:1;876;869:12;833:50;916:4;908:6;904:17;892:29;;968:3;961:4;952:6;944;940:19;936:30;933:39;930:59;;;985:1;982;975:12;930:59;648:347;;;;;:::o;1000:829::-;1167:6;1175;1183;1191;1199;1252:3;1240:9;1231:7;1227:23;1223:33;1220:53;;;1269:1;1266;1259:12;1220:53;1308:9;1295:23;1327:31;1352:5;1327:31;:::i;:::-;1377:5;-1:-1:-1;1401:63:89;1456:7;1451:2;1436:18;;1401:63;:::i;:::-;1391:73;;1483:78;1553:7;1547:3;1536:9;1532:19;1483:78;:::i;:::-;1473:88;;1612:3;1601:9;1597:19;1584:33;1640:18;1632:6;1629:30;1626:50;;;1672:1;1669;1662:12;1626:50;1711:58;1761:7;1752:6;1741:9;1737:22;1711:58;:::i;:::-;1000:829;;;;-1:-1:-1;1000:829:89;;-1:-1:-1;1788:8:89;;1685:84;1000:829;-1:-1:-1;;;1000:829:89:o;2041:815::-;2166:6;2174;2182;2190;2198;2251:3;2239:9;2230:7;2226:23;2222:33;2219:53;;;2268:1;2265;2258:12;2219:53;2307:9;2294:23;2326:31;2351:5;2326:31;:::i;:::-;2376:5;-1:-1:-1;2400:63:89;2455:7;2450:2;2435:18;;2400:63;:::i;:::-;2390:73;;2515:3;2504:9;2500:19;2487:33;2529;2554:7;2529:33;:::i;:::-;2581:7;-1:-1:-1;2639:3:89;2624:19;;2611:33;2667:18;2656:30;;2653:50;;;2699:1;2696;2689:12;3114:158;3177:5;3222:2;3213:6;3208:3;3204:16;3200:25;3197:45;;;3238:1;3235;3228:12;3277:807;3433:6;3441;3449;3457;3465;3518:3;3506:9;3497:7;3493:23;3489:33;3486:53;;;3535:1;3532;3525:12;3486:53;3574:9;3561:23;3593:31;3618:5;3593:31;:::i;:::-;3643:5;-1:-1:-1;3667:63:89;3722:7;3717:2;3702:18;;3667:63;:::i;:::-;3657:73;;3749:67;3808:7;3802:3;3791:9;3787:19;3749:67;:::i;:::-;3739:77;;3867:3;3856:9;3852:19;3839:33;3895:18;3887:6;3884:30;3881:50;;;3927:1;3924;3917:12;4487:932;4697:6;4705;4713;4721;4729;4737;4790:3;4778:9;4769:7;4765:23;4761:33;4758:53;;;4807:1;4804;4797:12;4758:53;4846:9;4833:23;4865:31;4890:5;4865:31;:::i;:::-;4915:5;-1:-1:-1;4939:63:89;4994:7;4989:2;4974:18;;4939:63;:::i;:::-;4929:73;;5021:78;5091:7;5085:3;5074:9;5070:19;5021:78;:::i;:::-;5011:88;;5146:3;5135:9;5131:19;5118:33;5108:43;;5202:3;5191:9;5187:19;5174:33;5230:18;5222:6;5219:30;5216:50;;;5262:1;5259;5252:12;5216:50;5301:58;5351:7;5342:6;5331:9;5327:22;5301:58;:::i;:::-;4487:932;;;;-1:-1:-1;4487:932:89;;-1:-1:-1;4487:932:89;;5378:8;;4487:932;-1:-1:-1;;;4487:932:89:o;5735:409::-;5805:6;5813;5866:2;5854:9;5845:7;5841:23;5837:32;5834:52;;;5882:1;5879;5872:12;5834:52;5922:9;5909:23;5955:18;5947:6;5944:30;5941:50;;;5987:1;5984;5977:12;5941:50;6026:58;6076:7;6067:6;6056:9;6052:22;6026:58;:::i;:::-;6103:8;;6000:84;;-1:-1:-1;5735:409:89;-1:-1:-1;;;;5735:409:89:o;6149:416::-;6296:2;6285:9;6278:21;6259:4;6328:6;6322:13;6371:6;6366:2;6355:9;6351:18;6344:34;6430:6;6425:2;6417:6;6413:15;6408:2;6397:9;6393:18;6387:50;6486:1;6481:2;6472:6;6461:9;6457:22;6453:31;6446:42;6556:2;6549;6545:7;6540:2;6532:6;6528:15;6524:29;6513:9;6509:45;6505:54;6497:62;;;6149:416;;;;:::o;6570:208::-;6657:6;6710:2;6698:9;6689:7;6685:23;6681:32;6678:52;;;6726:1;6723;6716:12;6678:52;-1:-1:-1;6749:23:89;;6570:208;-1:-1:-1;6570:208:89:o;6965:160::-;7031:20;;7091:1;7080:20;;;7070:31;;7060:59;;7115:1;7112;7105:12;7130:886;7262:6;7270;7278;7286;7294;7302;7355:3;7343:9;7334:7;7330:23;7326:33;7323:53;;;7372:1;7369;7362:12;7323:53;7411:9;7398:23;7430:31;7455:5;7430:31;:::i;:::-;7480:5;-1:-1:-1;7504:63:89;7559:7;7554:2;7539:18;;7504:63;:::i;:::-;7494:73;;7619:3;7608:9;7604:19;7591:33;7633;7658:7;7633:33;:::i;:::-;7685:7;-1:-1:-1;7711:37:89;7743:3;7728:19;;7711:37;:::i;:::-;7701:47;;7799:3;7788:9;7784:19;7771:33;7827:18;7819:6;7816:30;7813:50;;;7859:1;7856;7849:12;8021:910;8220:6;8228;8236;8244;8252;8260;8313:3;8301:9;8292:7;8288:23;8284:33;8281:53;;;8330:1;8327;8320:12;8281:53;8369:9;8356:23;8388:31;8413:5;8388:31;:::i;:::-;8438:5;-1:-1:-1;8462:63:89;8517:7;8512:2;8497:18;;8462:63;:::i;:::-;8452:73;;8544:67;8603:7;8597:3;8586:9;8582:19;8544:67;:::i;:::-;8534:77;;8658:3;8647:9;8643:19;8630:33;8620:43;;8714:3;8703:9;8699:19;8686:33;8742:18;8734:6;8731:30;8728:50;;;8774:1;8771;8764:12;9228:811;9362:6;9370;9378;9386;9394;9402;9455:3;9443:9;9434:7;9430:23;9426:33;9423:53;;;9472:1;9469;9462:12;9423:53;9511:9;9498:23;9530:31;9555:5;9530:31;:::i;:::-;9580:5;-1:-1:-1;9604:63:89;9659:7;9654:2;9639:18;;9604:63;:::i;:::-;9594:73;;9714:3;9703:9;9699:19;9686:33;9676:43;;9766:3;9755:9;9751:19;9738:33;9728:43;;9822:3;9811:9;9807:19;9794:33;9850:18;9842:6;9839:30;9836:50;;;9882:1;9879;9872:12;10140:1903;10363:13;;10114;10107:21;10095:34;;10334:3;10319:19;;10435:4;10427:6;10423:17;10417:24;10450:51;10495:4;10484:9;10480:20;10466:12;10114:13;10107:21;10095:34;;10044:91;10450:51;;10550:4;10542:6;10538:17;10532:24;10565:53;10612:4;10601:9;10597:20;10581:14;10114:13;10107:21;10095:34;;10044:91;10565:53;;10667:4;10659:6;10655:17;10649:24;10682:53;10729:4;10718:9;10714:20;10698:14;10114:13;10107:21;10095:34;;10044:91;10682:53;;10784:4;10776:6;10772:17;10766:24;10799:53;10846:4;10835:9;10831:20;10815:14;10114:13;10107:21;10095:34;;10044:91;10799:53;;10901:4;10893:6;10889:17;10883:24;10916:53;10963:4;10952:9;10948:20;10932:14;10114:13;10107:21;10095:34;;10044:91;10916:53;;11018:4;11010:6;11006:17;11000:24;11033:53;11080:4;11069:9;11065:20;11049:14;10114:13;10107:21;10095:34;;10044:91;11033:53;;11135:4;11127:6;11123:17;11117:24;11150:53;11197:4;11186:9;11182:20;11166:14;10114:13;10107:21;10095:34;;10044:91;11150:53;-1:-1:-1;11222:6:89;11265:15;;;11259:22;10114:13;10107:21;11322:18;;;10095:34;11360:6;11403:15;;;11397:22;10114:13;10107:21;11460:18;;;10095:34;11498:6;11541:15;;;11535:22;10114:13;10107:21;11598:18;;;10095:34;11636:6;11680:15;;;11674:22;10114:13;10107:21;11738:18;;;10095:34;11776:6;11820:15;;;11814:22;10114:13;10107:21;11878:18;;;10095:34;11916:6;11960:15;;;11954:22;10114:13;10107:21;12018:18;;;;10095:34;;;;10140:1903;:::o;12048:161::-;12115:20;;12175:8;12164:20;;12154:31;;12144:59;;12199:1;12196;12189:12;12214:959;12299:6;12352:3;12340:9;12331:7;12327:23;12323:33;12320:53;;;12369:1;12366;12359:12;12320:53;12402:2;12396:9;12444:3;12436:6;12432:16;12514:6;12502:10;12499:22;12478:18;12466:10;12463:34;12460:62;12457:185;;;12564:10;12559:3;12555:20;12552:1;12545:31;12599:4;12596:1;12589:15;12627:4;12624:1;12617:15;12457:185;12658:2;12651:22;12695:23;;12727:31;12695:23;12727:31;:::i;:::-;12767:21;;12840:2;12825:18;;12812:32;12853:33;12812:32;12853:33;:::i;:::-;12914:2;12902:15;;12895:32;12960:37;12993:2;12978:18;;12960:37;:::i;:::-;12955:2;12947:6;12943:15;12936:62;13031:36;13063:2;13052:9;13048:18;13031:36;:::i;:::-;13026:2;13018:6;13014:15;13007:61;13102:39;13136:3;13125:9;13121:19;13102:39;:::i;:::-;13096:3;13084:16;;13077:65;13088:6;12214:959;-1:-1:-1;;;12214:959:89:o;13178:232::-;13217:3;13238:17;;;13235:140;;13297:10;13292:3;13288:20;13285:1;13278:31;13332:4;13329:1;13322:15;13360:4;13357:1;13350:15;13235:140;-1:-1:-1;13402:1:89;13391:13;;13178:232::o;13415:271::-;13598:6;13590;13585:3;13572:33;13554:3;13624:16;;13649:13;;;13624:16;13415:271;-1:-1:-1;13415:271:89:o","linkReferences":{},"immutableReferences":{"598":[{"start":301,"length":32},{"start":1024,"length":32}]}},"methodIdentifiers":{"afterAddLiquidity(address,(address,address,uint24,int24,address),(int24,int24,int256,bytes32),int256,bytes)":"5a2a8100","afterDonate(address,(address,address,uint24,int24,address),uint256,uint256,bytes)":"e1b4af69","afterInitialize(address,(address,address,uint24,int24,address),uint160,int24,bytes)":"a910f80f","afterRemoveLiquidity(address,(address,address,uint24,int24,address),(int24,int24,int256,bytes32),int256,bytes)":"8db2b652","afterSwap(address,(address,address,uint24,int24,address),(bool,int256,uint160),int256,bytes)":"b47b2fb1","beforeAddLiquidity(address,(address,address,uint24,int24,address),(int24,int24,int256,bytes32),bytes)":"259982e5","beforeDonate(address,(address,address,uint24,int24,address),uint256,uint256,bytes)":"b6a8b0fa","beforeInitialize(address,(address,address,uint24,int24,address),uint160,bytes)":"3440d820","beforeRemoveLiquidity(address,(address,address,uint24,int24,address),(int24,int24,int256,bytes32),bytes)":"21d0ee70","beforeSwap(address,(address,address,uint24,int24,address),(bool,int256,uint160),bytes)":"575e24b4","getHookPermissions()":"c4e833ce","manager()":"481c6a75","swapCount(bytes32)":"94168af8","unlockCallback(bytes)":"91dd7346"},"rawMetadata":"{\"compiler\":{\"version\":\"0.8.25+commit.b61c2a91\"},\"language\":\"Solidity\",\"output\":{\"abi\":[{\"inputs\":[{\"internalType\":\"contract IPoolManager\",\"name\":\"_poolManager\",\"type\":\"address\"}],\"stateMutability\":\"nonpayable\",\"type\":\"constructor\"},{\"inputs\":[],\"name\":\"HookNotImplemented\",\"type\":\"error\"},{\"inputs\":[],\"name\":\"InvalidPool\",\"type\":\"error\"},{\"inputs\":[],\"name\":\"LockFailure\",\"type\":\"error\"},{\"inputs\":[],\"name\":\"NotManager\",\"type\":\"error\"},{\"inputs\":[],\"name\":\"NotSelf\",\"type\":\"error\"},{\"inputs\":[{\"internalType\":\"address\",\"name\":\"\",\"type\":\"address\"},{\"components\":[{\"internalType\":\"Currency\",\"name\":\"currency0\",\"type\":\"address\"},{\"internalType\":\"Currency\",\"name\":\"currency1\",\"type\":\"address\"},{\"internalType\":\"uint24\",\"name\":\"fee\",\"type\":\"uint24\"},{\"internalType\":\"int24\",\"name\":\"tickSpacing\",\"type\":\"int24\"},{\"internalType\":\"contract IHooks\",\"name\":\"hooks\",\"type\":\"address\"}],\"internalType\":\"struct PoolKey\",\"name\":\"\",\"type\":\"tuple\"},{\"components\":[{\"internalType\":\"int24\",\"name\":\"tickLower\",\"type\":\"int24\"},{\"internalType\":\"int24\",\"name\":\"tickUpper\",\"type\":\"int24\"},{\"internalType\":\"int256\",\"name\":\"liquidityDelta\",\"type\":\"int256\"},{\"internalType\":\"bytes32\",\"name\":\"salt\",\"type\":\"bytes32\"}],\"internalType\":\"struct IPoolManager.ModifyLiquidityParams\",\"name\":\"\",\"type\":\"tuple\"},{\"internalType\":\"BalanceDelta\",\"name\":\"\",\"type\":\"int256\"},{\"internalType\":\"bytes\",\"name\":\"\",\"type\":\"bytes\"}],\"name\":\"afterAddLiquidity\",\"outputs\":[{\"internalType\":\"bytes4\",\"name\":\"\",\"type\":\"bytes4\"},{\"internalType\":\"BalanceDelta\",\"name\":\"\",\"type\":\"int256\"}],\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"inputs\":[{\"internalType\":\"address\",\"name\":\"\",\"type\":\"address\"},{\"components\":[{\"internalType\":\"Currency\",\"name\":\"currency0\",\"type\":\"address\"},{\"internalType\":\"Currency\",\"name\":\"currency1\",\"type\":\"address\"},{\"internalType\":\"uint24\",\"name\":\"fee\",\"type\":\"uint24\"},{\"internalType\":\"int24\",\"name\":\"tickSpacing\",\"type\":\"int24\"},{\"internalType\":\"contract IHooks\",\"name\":\"hooks\",\"type\":\"address\"}],\"internalType\":\"struct PoolKey\",\"name\":\"\",\"type\":\"tuple\"},{\"internalType\":\"uint256\",\"name\":\"\",\"type\":\"uint256\"},{\"internalType\":\"uint256\",\"name\":\"\",\"type\":\"uint256\"},{\"internalType\":\"bytes\",\"name\":\"\",\"type\":\"bytes\"}],\"name\":\"afterDonate\",\"outputs\":[{\"internalType\":\"bytes4\",\"name\":\"\",\"type\":\"bytes4\"}],\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"inputs\":[{\"internalType\":\"address\",\"name\":\"\",\"type\":\"address\"},{\"components\":[{\"internalType\":\"Currency\",\"name\":\"currency0\",\"type\":\"address\"},{\"internalType\":\"Currency\",\"name\":\"currency1\",\"type\":\"address\"},{\"internalType\":\"uint24\",\"name\":\"fee\",\"type\":\"uint24\"},{\"internalType\":\"int24\",\"name\":\"tickSpacing\",\"type\":\"int24\"},{\"internalType\":\"contract IHooks\",\"name\":\"hooks\",\"type\":\"address\"}],\"internalType\":\"struct PoolKey\",\"name\":\"\",\"type\":\"tuple\"},{\"internalType\":\"uint160\",\"name\":\"\",\"type\":\"uint160\"},{\"internalType\":\"int24\",\"name\":\"\",\"type\":\"int24\"},{\"internalType\":\"bytes\",\"name\":\"\",\"type\":\"bytes\"}],\"name\":\"afterInitialize\",\"outputs\":[{\"internalType\":\"bytes4\",\"name\":\"\",\"type\":\"bytes4\"}],\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"inputs\":[{\"internalType\":\"address\",\"name\":\"\",\"type\":\"address\"},{\"components\":[{\"internalType\":\"Currency\",\"name\":\"currency0\",\"type\":\"address\"},{\"internalType\":\"Currency\",\"name\":\"currency1\",\"type\":\"address\"},{\"internalType\":\"uint24\",\"name\":\"fee\",\"type\":\"uint24\"},{\"internalType\":\"int24\",\"name\":\"tickSpacing\",\"type\":\"int24\"},{\"internalType\":\"contract IHooks\",\"name\":\"hooks\",\"type\":\"address\"}],\"internalType\":\"struct PoolKey\",\"name\":\"\",\"type\":\"tuple\"},{\"components\":[{\"internalType\":\"int24\",\"name\":\"tickLower\",\"type\":\"int24\"},{\"internalType\":\"int24\",\"name\":\"tickUpper\",\"type\":\"int24\"},{\"internalType\":\"int256\",\"name\":\"liquidityDelta\",\"type\":\"int256\"},{\"internalType\":\"bytes32\",\"name\":\"salt\",\"type\":\"bytes32\"}],\"internalType\":\"struct IPoolManager.ModifyLiquidityParams\",\"name\":\"\",\"type\":\"tuple\"},{\"internalType\":\"BalanceDelta\",\"name\":\"\",\"type\":\"int256\"},{\"internalType\":\"bytes\",\"name\":\"\",\"type\":\"bytes\"}],\"name\":\"afterRemoveLiquidity\",\"outputs\":[{\"internalType\":\"bytes4\",\"name\":\"\",\"type\":\"bytes4\"},{\"internalType\":\"BalanceDelta\",\"name\":\"\",\"type\":\"int256\"}],\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"inputs\":[{\"internalType\":\"address\",\"name\":\"\",\"type\":\"address\"},{\"components\":[{\"internalType\":\"Currency\",\"name\":\"currency0\",\"type\":\"address\"},{\"internalType\":\"Currency\",\"name\":\"currency1\",\"type\":\"address\"},{\"internalType\":\"uint24\",\"name\":\"fee\",\"type\":\"uint24\"},{\"internalType\":\"int24\",\"name\":\"tickSpacing\",\"type\":\"int24\"},{\"internalType\":\"contract IHooks\",\"name\":\"hooks\",\"type\":\"address\"}],\"internalType\":\"struct PoolKey\",\"name\":\"\",\"type\":\"tuple\"},{\"components\":[{\"internalType\":\"bool\",\"name\":\"zeroForOne\",\"type\":\"bool\"},{\"internalType\":\"int256\",\"name\":\"amountSpecified\",\"type\":\"int256\"},{\"internalType\":\"uint160\",\"name\":\"sqrtPriceLimitX96\",\"type\":\"uint160\"}],\"internalType\":\"struct IPoolManager.SwapParams\",\"name\":\"\",\"type\":\"tuple\"},{\"internalType\":\"BalanceDelta\",\"name\":\"\",\"type\":\"int256\"},{\"internalType\":\"bytes\",\"name\":\"\",\"type\":\"bytes\"}],\"name\":\"afterSwap\",\"outputs\":[{\"internalType\":\"bytes4\",\"name\":\"\",\"type\":\"bytes4\"},{\"internalType\":\"int128\",\"name\":\"\",\"type\":\"int128\"}],\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"inputs\":[{\"internalType\":\"address\",\"name\":\"\",\"type\":\"address\"},{\"components\":[{\"internalType\":\"Currency\",\"name\":\"currency0\",\"type\":\"address\"},{\"internalType\":\"Currency\",\"name\":\"currency1\",\"type\":\"address\"},{\"internalType\":\"uint24\",\"name\":\"fee\",\"type\":\"uint24\"},{\"internalType\":\"int24\",\"name\":\"tickSpacing\",\"type\":\"int24\"},{\"internalType\":\"contract IHooks\",\"name\":\"hooks\",\"type\":\"address\"}],\"internalType\":\"struct PoolKey\",\"name\":\"\",\"type\":\"tuple\"},{\"components\":[{\"internalType\":\"int24\",\"name\":\"tickLower\",\"type\":\"int24\"},{\"internalType\":\"int24\",\"name\":\"tickUpper\",\"type\":\"int24\"},{\"internalType\":\"int256\",\"name\":\"liquidityDelta\",\"type\":\"int256\"},{\"internalType\":\"bytes32\",\"name\":\"salt\",\"type\":\"bytes32\"}],\"internalType\":\"struct IPoolManager.ModifyLiquidityParams\",\"name\":\"\",\"type\":\"tuple\"},{\"internalType\":\"bytes\",\"name\":\"\",\"type\":\"bytes\"}],\"name\":\"beforeAddLiquidity\",\"outputs\":[{\"internalType\":\"bytes4\",\"name\":\"\",\"type\":\"bytes4\"}],\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"inputs\":[{\"internalType\":\"address\",\"name\":\"\",\"type\":\"address\"},{\"components\":[{\"internalType\":\"Currency\",\"name\":\"currency0\",\"type\":\"address\"},{\"internalType\":\"Currency\",\"name\":\"currency1\",\"type\":\"address\"},{\"internalType\":\"uint24\",\"name\":\"fee\",\"type\":\"uint24\"},{\"internalType\":\"int24\",\"name\":\"tickSpacing\",\"type\":\"int24\"},{\"internalType\":\"contract IHooks\",\"name\":\"hooks\",\"type\":\"address\"}],\"internalType\":\"struct PoolKey\",\"name\":\"\",\"type\":\"tuple\"},{\"internalType\":\"uint256\",\"name\":\"\",\"type\":\"uint256\"},{\"internalType\":\"uint256\",\"name\":\"\",\"type\":\"uint256\"},{\"internalType\":\"bytes\",\"name\":\"\",\"type\":\"bytes\"}],\"name\":\"beforeDonate\",\"outputs\":[{\"internalType\":\"bytes4\",\"name\":\"\",\"type\":\"bytes4\"}],\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"inputs\":[{\"internalType\":\"address\",\"name\":\"\",\"type\":\"address\"},{\"components\":[{\"internalType\":\"Currency\",\"name\":\"currency0\",\"type\":\"address\"},{\"internalType\":\"Currency\",\"name\":\"currency1\",\"type\":\"address\"},{\"internalType\":\"uint24\",\"name\":\"fee\",\"type\":\"uint24\"},{\"internalType\":\"int24\",\"name\":\"tickSpacing\",\"type\":\"int24\"},{\"internalType\":\"contract IHooks\",\"name\":\"hooks\",\"type\":\"address\"}],\"internalType\":\"struct PoolKey\",\"name\":\"\",\"type\":\"tuple\"},{\"internalType\":\"uint160\",\"name\":\"\",\"type\":\"uint160\"},{\"internalType\":\"bytes\",\"name\":\"\",\"type\":\"bytes\"}],\"name\":\"beforeInitialize\",\"outputs\":[{\"internalType\":\"bytes4\",\"name\":\"\",\"type\":\"bytes4\"}],\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"inputs\":[{\"internalType\":\"address\",\"name\":\"\",\"type\":\"address\"},{\"components\":[{\"internalType\":\"Currency\",\"name\":\"currency0\",\"type\":\"address\"},{\"internalType\":\"Currency\",\"name\":\"currency1\",\"type\":\"address\"},{\"internalType\":\"uint24\",\"name\":\"fee\",\"type\":\"uint24\"},{\"internalType\":\"int24\",\"name\":\"tickSpacing\",\"type\":\"int24\"},{\"internalType\":\"contract IHooks\",\"name\":\"hooks\",\"type\":\"address\"}],\"internalType\":\"struct PoolKey\",\"name\":\"\",\"type\":\"tuple\"},{\"components\":[{\"internalType\":\"int24\",\"name\":\"tickLower\",\"type\":\"int24\"},{\"internalType\":\"int24\",\"name\":\"tickUpper\",\"type\":\"int24\"},{\"internalType\":\"int256\",\"name\":\"liquidityDelta\",\"type\":\"int256\"},{\"internalType\":\"bytes32\",\"name\":\"salt\",\"type\":\"bytes32\"}],\"internalType\":\"struct IPoolManager.ModifyLiquidityParams\",\"name\":\"\",\"type\":\"tuple\"},{\"internalType\":\"bytes\",\"name\":\"\",\"type\":\"bytes\"}],\"name\":\"beforeRemoveLiquidity\",\"outputs\":[{\"internalType\":\"bytes4\",\"name\":\"\",\"type\":\"bytes4\"}],\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"inputs\":[{\"internalType\":\"address\",\"name\":\"\",\"type\":\"address\"},{\"components\":[{\"internalType\":\"Currency\",\"name\":\"currency0\",\"type\":\"address\"},{\"internalType\":\"Currency\",\"name\":\"currency1\",\"type\":\"address\"},{\"internalType\":\"uint24\",\"name\":\"fee\",\"type\":\"uint24\"},{\"internalType\":\"int24\",\"name\":\"tickSpacing\",\"type\":\"int24\"},{\"internalType\":\"contract IHooks\",\"name\":\"hooks\",\"type\":\"address\"}],\"internalType\":\"struct PoolKey\",\"name\":\"key\",\"type\":\"tuple\"},{\"components\":[{\"internalType\":\"bool\",\"name\":\"zeroForOne\",\"type\":\"bool\"},{\"internalType\":\"int256\",\"name\":\"amountSpecified\",\"type\":\"int256\"},{\"internalType\":\"uint160\",\"name\":\"sqrtPriceLimitX96\",\"type\":\"uint160\"}],\"internalType\":\"struct IPoolManager.SwapParams\",\"name\":\"\",\"type\":\"tuple\"},{\"internalType\":\"bytes\",\"name\":\"\",\"type\":\"bytes\"}],\"name\":\"beforeSwap\",\"outputs\":[{\"internalType\":\"bytes4\",\"name\":\"\",\"type\":\"bytes4\"},{\"internalType\":\"BeforeSwapDelta\",\"name\":\"\",\"type\":\"int256\"},{\"internalType\":\"uint24\",\"name\":\"\",\"type\":\"uint24\"}],\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"inputs\":[],\"name\":\"getHookPermissions\",\"outputs\":[{\"components\":[{\"internalType\":\"bool\",\"name\":\"beforeInitialize\",\"type\":\"bool\"},{\"internalType\":\"bool\",\"name\":\"afterInitialize\",\"type\":\"bool\"},{\"internalType\":\"bool\",\"name\":\"beforeAddLiquidity\",\"type\":\"bool\"},{\"internalType\":\"bool\",\"name\":\"afterAddLiquidity\",\"type\":\"bool\"},{\"internalType\":\"bool\",\"name\":\"beforeRemoveLiquidity\",\"type\":\"bool\"},{\"internalType\":\"bool\",\"name\":\"afterRemoveLiquidity\",\"type\":\"bool\"},{\"internalType\":\"bool\",\"name\":\"beforeSwap\",\"type\":\"bool\"},{\"internalType\":\"bool\",\"name\":\"afterSwap\",\"type\":\"bool\"},{\"internalType\":\"bool\",\"name\":\"beforeDonate\",\"type\":\"bool\"},{\"internalType\":\"bool\",\"name\":\"afterDonate\",\"type\":\"bool\"},{\"internalType\":\"bool\",\"name\":\"beforeSwapReturnDelta\",\"type\":\"bool\"},{\"internalType\":\"bool\",\"name\":\"afterSwapReturnDelta\",\"type\":\"bool\"},{\"internalType\":\"bool\",\"name\":\"afterAddLiquidityReturnDelta\",\"type\":\"bool\"},{\"internalType\":\"bool\",\"name\":\"afterRemoveLiquidityReturnDelta\",\"type\":\"bool\"}],\"internalType\":\"struct Hooks.Permissions\",\"name\":\"\",\"type\":\"tuple\"}],\"stateMutability\":\"pure\",\"type\":\"function\"},{\"inputs\":[],\"name\":\"manager\",\"outputs\":[{\"internalType\":\"contract IPoolManager\",\"name\":\"\",\"type\":\"address\"}],\"stateMutability\":\"view\",\"type\":\"function\"},{\"inputs\":[{\"internalType\":\"PoolId\",\"name\":\"\",\"type\":\"bytes32\"}],\"name\":\"swapCount\",\"outputs\":[{\"internalType\":\"uint256\",\"name\":\"\",\"type\":\"uint256\"}],\"stateMutability\":\"view\",\"type\":\"function\"},{\"inputs\":[{\"internalType\":\"bytes\",\"name\":\"data\",\"type\":\"bytes\"}],\"name\":\"unlockCallback\",\"outputs\":[{\"internalType\":\"bytes\",\"name\":\"\",\"type\":\"bytes\"}],\"stateMutability\":\"nonpayable\",\"type\":\"function\"}],\"devdoc\":{\"kind\":\"dev\",\"methods\":{\"unlockCallback(bytes)\":{\"details\":\"We force the onlyByManager modifier by exposing a virtual function after the onlyByManager check.\"}},\"version\":1},\"userdoc\":{\"kind\":\"user\",\"methods\":{},\"version\":1}},\"settings\":{\"compilationTarget\":{\"src/generated/test_1.sol\":\"SimpleCounterHook\"},\"evmVersion\":\"cancun\",\"libraries\":{},\"metadata\":{\"bytecodeHash\":\"ipfs\"},\"optimizer\":{\"enabled\":true,\"runs\":800},\"remappings\":[\":@axiom-crypto/v2-periphery/=lib/axiom-std/lib/axiom-v2-periphery/src/\",\":@ensdomains/=lib/v4-periphery/lib/v4-core/node_modules/@ensdomains/\",\":@openzeppelin/=lib/v4-periphery/lib/v4-core/lib/openzeppelin-contracts/\",\":@uniswap/v4-core/=lib/v4-periphery/lib/v4-core/\",\":axiom-std/=lib/axiom-std/src/\",\":axiom-v2-periphery/=lib/axiom-std/lib/axiom-v2-periphery/\",\":ds-test/=lib/v4-periphery/lib/forge-std/lib/ds-test/src/\",\":erc4626-tests/=lib/v4-periphery/lib/openzeppelin-contracts/lib/erc4626-tests/\",\":forge-gas-snapshot/=lib/v4-periphery/lib/forge-gas-snapshot/src/\",\":forge-std/=lib/forge-std/src/\",\":hardhat/=lib/v4-periphery/lib/v4-core/node_modules/hardhat/\",\":openzeppelin-contracts/=lib/v4-periphery/lib/v4-core/lib/openzeppelin-contracts/\",\":openzeppelin/=lib/v4-periphery/lib/openzeppelin-contracts/contracts/\",\":solmate/=lib/v4-periphery/lib/solmate/src/\",\":v4-core/=lib/v4-periphery/lib/v4-core/\",\":v4-periphery/=lib/v4-periphery/contracts/\"]},\"sources\":{\"lib/v4-periphery/contracts/BaseHook.sol\":{\"keccak256\":\"0x799f21eb68a0f64398c6cb63f4911a06c471796d2bb340bb16943195a55c1085\",\"license\":\"UNLICENSED\",\"urls\":[\"bzz-raw://dcdd6f7d8a8c7d4b6f297ed2b81539d14fa84dc9fab09732c83f6a0c0dcf4651\",\"dweb:/ipfs/QmQoXjxGmDgDfKNjdHBT4m6zNUGcB35CZQBrZK9VBJhydK\"]},\"lib/v4-periphery/contracts/base/ImmutableState.sol\":{\"keccak256\":\"0xef7c9afd38504de0b5c1f0489cebbf100d3c6f71f2035d1659c8ae90dafe4e99\",\"license\":\"GPL-2.0-or-later\",\"urls\":[\"bzz-raw://4f73fc15f80565debf95f184e6cbb4cbefe775eb6b615651b0e9dd161ce541b9\",\"dweb:/ipfs/Qmdu4FpSNzujoafXEKeUZnArX4NUqzdRcrnM8bTAXBHToC\"]},\"lib/v4-periphery/contracts/base/SafeCallback.sol\":{\"keccak256\":\"0x2a73d4104f47cf6b81ecfe6713b2c59135ef1821dd13b242d26b3bd7239ee78f\",\"license\":\"UNLICENSED\",\"urls\":[\"bzz-raw://fe07ea01d76fe78f19def57a409960fc00c122715cba310665e31ffa052ed048\",\"dweb:/ipfs/QmSpQTs3VUAQgfGuKpZ94gk2jQT86aWw27HAc1NCdTkMSX\"]},\"lib/v4-periphery/lib/v4-core/src/interfaces/IExtsload.sol\":{\"keccak256\":\"0x5457e68d4c21dd247bf5e3bea59df3372b197afb1856130b3512e101269742c9\",\"license\":\"MIT\",\"urls\":[\"bzz-raw://ec7ebfb1bea371ec76f2d8d9cdb5729176cd56432f20284042c8bd303c9b6221\",\"dweb:/ipfs/QmaP2NLoyYmB1nxEU3RAEP7Q6AJBdLvjaGgfihLVXcuWgz\"]},\"lib/v4-periphery/lib/v4-core/src/interfaces/IExttload.sol\":{\"keccak256\":\"0xc70fcba06b34aa85a53476b251c60a72272ebf83784e39fd9f0a3a75977585a3\",\"license\":\"MIT\",\"urls\":[\"bzz-raw://b20481fe84db25f9bb896bb7afd1e6a6dfc70b6d4a67d9d1f282fb1214bd2f10\",\"dweb:/ipfs/QmNhDbqgSdhx9Bbtifk5YX6Dpj8yKKKJf75y7JijYPDrF1\"]},\"lib/v4-periphery/lib/v4-core/src/interfaces/IHooks.sol\":{\"keccak256\":\"0x8751728dcb4923b86dbd7523e7a5a515177255bf64d6d95daaa00d510e97fb75\",\"license\":\"MIT\",\"urls\":[\"bzz-raw://92e9a4932961ec52fbca3fc3dd1704ac9c30541cfea7884c1d00a44632102bca\",\"dweb:/ipfs/QmZWFtusFQLK9kXJLGWEMEHgV8LtvX9cKomqncF2mscJro\"]},\"lib/v4-periphery/lib/v4-core/src/interfaces/IPoolManager.sol\":{\"keccak256\":\"0xcfc4084f2519a4768a974a2b3a577866949674e76e10cc55f0e00ec6a910d38d\",\"license\":\"MIT\",\"urls\":[\"bzz-raw://769a40fded9d60d41a1443ad7a33b92972d7d0de8e93325ee90de6a35e2d85de\",\"dweb:/ipfs/Qmbp5sPXTBDaRTJq5cggW7JpPGvNp3REkFUAS2dxh4xcwn\"]},\"lib/v4-periphery/lib/v4-core/src/interfaces/IProtocolFeeController.sol\":{\"keccak256\":\"0x0a48b2b660ef63aa520640bd7a80fdf7cf1fd19a268f155b029e5ca31b8ae4a6\",\"license\":\"MIT\",\"urls\":[\"bzz-raw://a2a74397cc05a7c32347c2bc63ab7dd4a7b436c7bb48b7e1910bc33a550f8116\",\"dweb:/ipfs/QmXd91nmFT87pZc5DkZD4wQ6E2exLNAMkm2qCrCBN12de3\"]},\"lib/v4-periphery/lib/v4-core/src/interfaces/IProtocolFees.sol\":{\"keccak256\":\"0x78eb297b0e978d1b8217292a818d69ecfc403b4a3562d2eb9dfd7f1b781dc38b\",\"license\":\"MIT\",\"urls\":[\"bzz-raw://9abdc8e151e8bceb40d3667fc0345cd69b7379725f720490401ad3775527bf68\",\"dweb:/ipfs/QmNoHFjChTCoC4a2TvomtGssddq7j2Rga4mrSwFm1VQ6My\"]},\"lib/v4-periphery/lib/v4-core/src/interfaces/callback/IUnlockCallback.sol\":{\"keccak256\":\"0xe413ff595dadedfeff672247031151c309f15a104b7ed03a51400e50c337d373\",\"license\":\"MIT\",\"urls\":[\"bzz-raw://1558caec33b3af1e3b8080973cd1f7f2e4f1483b06c31dbb84cdc2414b1f5a1e\",\"dweb:/ipfs/QmYgQXpMTvb3ZJex1s61QKZQ9WRRqFg6HRiYndgPh7yKT1\"]},\"lib/v4-periphery/lib/v4-core/src/interfaces/external/IERC20Minimal.sol\":{\"keccak256\":\"0x8dc5ee72426906aeb12f4ae6a2426df059491739bd5cbdb8ea4a3e26970436c6\",\"license\":\"GPL-2.0-or-later\",\"urls\":[\"bzz-raw://f7e30dbbc972cb78d169c8845d6210d27490c8534e68a20d4f296b11ec727b92\",\"dweb:/ipfs/QmXBvaWaAnPW25YGVm2sMbf5mwrzLapUcqPnrNiiGGXxmA\"]},\"lib/v4-periphery/lib/v4-core/src/interfaces/external/IERC6909Claims.sol\":{\"keccak256\":\"0x1d53aa307e997fd73e1a31abe1052aad574994627958399375f760b8a750afbc\",\"license\":\"MIT\",\"urls\":[\"bzz-raw://c308d1aba00f3f834357cea75e03a64eba3da425eb31ed6bd4a9e56ada606b79\",\"dweb:/ipfs/Qmb8fuGxpw1JRRFdXUF9U2Una6msqYFTXMvw6ULPHpJ4ow\"]},\"lib/v4-periphery/lib/v4-core/src/libraries/BitMath.sol\":{\"keccak256\":\"0x652cd36970d953b3067c766b3b0c0063d4c403eba7352df09ec10dcc868910d9\",\"license\":\"GPL-2.0-or-later\",\"urls\":[\"bzz-raw://e357b092171edb755c8514b4b4ae758a8b43458c7a23409ebf5afd325fefa2be\",\"dweb:/ipfs/QmWKRWT2z15cVbVVDkjqdVnpf8Ck1jytcXbA7zCUevp7b6\"]},\"lib/v4-periphery/lib/v4-core/src/libraries/CustomRevert.sol\":{\"keccak256\":\"0xd768d9e1ebdaa14b723a1bd4faf55c37104fa28c00bb8ee399dc8d55f0da480c\",\"license\":\"MIT\",\"urls\":[\"bzz-raw://683906f0281596df504b8b711ca8d1d48f65e03f4dd649802118e03469ebe421\",\"dweb:/ipfs/QmXcpr5RNefsAd33hmUDjbRVy2mRRrQtPHbDE3kYnLx4KR\"]},\"lib/v4-periphery/lib/v4-core/src/libraries/FixedPoint128.sol\":{\"keccak256\":\"0x1f015a7a107f4c471985a958f1bfbc1710948f9bcd3786a02310bc0ad5080c69\",\"license\":\"GPL-2.0-or-later\",\"urls\":[\"bzz-raw://a1cacac08675284f319d151f6bc5d1c66cf1d54c5dad1c620a2e1e1336cbb93a\",\"dweb:/ipfs/QmPf8vQ3mBqniKHwvS1DDEFrV5dJfZSL6mMi9JVy6omQaz\"]},\"lib/v4-periphery/lib/v4-core/src/libraries/FixedPoint96.sol\":{\"keccak256\":\"0x7731aeeba692b0ca5dac8f27803ec9e5ce98795a554c09ae05c220861bd0314e\",\"license\":\"GPL-2.0-or-later\",\"urls\":[\"bzz-raw://963a8155e442d31c4541c387d333db0c02941ae3517e9173ad30ebcb4f6a14e9\",\"dweb:/ipfs/QmevAr8oG4WSoi5gBqQ53VzBuFbDKoZHfJwjuTmVhzm8Tc\"]},\"lib/v4-periphery/lib/v4-core/src/libraries/FullMath.sol\":{\"keccak256\":\"0x480e1e2741db0d1d5843c81d25576f182f64db9b92788ccbfc321a05d726f5e3\",\"license\":\"MIT\",\"urls\":[\"bzz-raw://f5c9606989f6487189a70e383f88ec0b203f8232c6e901e51e421113008959fd\",\"dweb:/ipfs/QmeHAmfavKjLtcd1J53Bs9WbknH3jLZfmU9NeaUZCHCJwY\"]},\"lib/v4-periphery/lib/v4-core/src/libraries/Hooks.sol\":{\"keccak256\":\"0x036b3d7f46a6bc2f5d4d563a926baad015cb76f041b743af1a343b3f418a57cf\",\"license\":\"MIT\",\"urls\":[\"bzz-raw://f1e555dd1854d4c390b750ae23fc36818c0e2237f562fdcfd8404241825234b8\",\"dweb:/ipfs/QmcV3zQrydELX8G3nJRVLnA9SG5jar1uJejww9jbvuQ3cm\"]},\"lib/v4-periphery/lib/v4-core/src/libraries/LPFeeLibrary.sol\":{\"keccak256\":\"0x33f69b500fd414cdb9856d9e623d085f8a15a8d3a9570ac6339514c800d077bf\",\"license\":\"GPL-2.0-or-later\",\"urls\":[\"bzz-raw://c45ed697cee974ecd23cdd1995fb837cf6e8a9b18a57c1d5edfbd0e60a61e68c\",\"dweb:/ipfs/QmTNrppHysZU3WLt4TtFJmQ9CRbVUbk2Nfu6qoKiVCom2P\"]},\"lib/v4-periphery/lib/v4-core/src/libraries/LiquidityMath.sol\":{\"keccak256\":\"0xf60e2d395e715919a6cb0111dde08055a5983dce63dfdeef345b33f9a83a63fe\",\"license\":\"GPL-2.0-or-later\",\"urls\":[\"bzz-raw://fdbcf1d6df9c0ca34e133ca7fd90f6aba7bed873d397a07cd8c4e37bd69c23d8\",\"dweb:/ipfs/QmPMLysVr1yn5ZYMbC6g1MHL8RC4RCmk6omZzTePHJ9Kte\"]},\"lib/v4-periphery/lib/v4-core/src/libraries/ParseBytes.sol\":{\"keccak256\":\"0x6ce53c8138982fed86ce0a0f53c6859e1102c2b3105201320364d1f2f5f5d555\",\"license\":\"GPL-2.0-or-later\",\"urls\":[\"bzz-raw://b5891c8a94fad51a98764adea8940a8f13275fe0732d58bb1480c2405846a9e6\",\"dweb:/ipfs/QmPB31BvQ8DgaUjj1iSUdJxLMtMJhfMSEvnNKiQs18kTdq\"]},\"lib/v4-periphery/lib/v4-core/src/libraries/Pool.sol\":{\"keccak256\":\"0x5872d9930358dba79a19efacdf3993b541be408fbd8c301c23f0e4b344d2cec2\",\"license\":\"BUSL-1.1\",\"urls\":[\"bzz-raw://db6532f4eb1052a9df508d90ff070f67581c83be13e2e4a03062392ff22559a1\",\"dweb:/ipfs/QmSJX5Xzh38Zq1pGVmmS2239LocRdeYMmxi7TiEdeLzpUo\"]},\"lib/v4-periphery/lib/v4-core/src/libraries/Position.sol\":{\"keccak256\":\"0x3f99796bda1592c6df7140c7f6341bedda3458f785f86455b8e147bd6fafeafa\",\"license\":\"BUSL-1.1\",\"urls\":[\"bzz-raw://5c9c2aebfb7dee1971168c01cb31704dbb5645249cbd185353f9f04283ddd797\",\"dweb:/ipfs/QmQe9hngtfaHpEcroUGxWqLp6ceWaD5rHv1FffqK6XJpj3\"]},\"lib/v4-periphery/lib/v4-core/src/libraries/ProtocolFeeLibrary.sol\":{\"keccak256\":\"0xfe57793436b42e73edaf9281dbf170a03169e17123916dcaedc6bf6c63ed85cd\",\"license\":\"GPL-2.0-or-later\",\"urls\":[\"bzz-raw://be1430384e13155a63ae88732b6f562fe482b3d182bba352f5ae2d17d2edd382\",\"dweb:/ipfs/QmeKZZNByL2Z6f1mJYZeztjkFKYddYpTQKkPuMMj9M67e2\"]},\"lib/v4-periphery/lib/v4-core/src/libraries/SafeCast.sol\":{\"keccak256\":\"0x3dbf539c83fbda8bf5373f6c66b720ee4df1725e7e56b478bd938ea6cea4faf1\",\"license\":\"GPL-2.0-or-later\",\"urls\":[\"bzz-raw://b18b54bc7503a5fcd65cf7473778e17de582b8ad994e620ec1101791caf96eac\",\"dweb:/ipfs/QmVyh8EbAutw3F8vBPKBjLgoNJszqqb58zRzjVG64mydk6\"]},\"lib/v4-periphery/lib/v4-core/src/libraries/SqrtPriceMath.sol\":{\"keccak256\":\"0x7117071dc1c13358a28d1a8cdef483f62549e58c858fcdf734ddcb8545d40eae\",\"license\":\"GPL-2.0-or-later\",\"urls\":[\"bzz-raw://c8b93a915cc7fd9cd1e850555a0bd757b8fc33b99dbd39b2ab87930d72392620\",\"dweb:/ipfs/QmcMHNmqotZM3iFRSHDqwvq7cDtk8DHuQLtNgu93PPUvgE\"]},\"lib/v4-periphery/lib/v4-core/src/libraries/SwapMath.sol\":{\"keccak256\":\"0xf0fc6e250b4f3d77d91ca2cb7ad26d0c4753656a5f567db53a602fbbe7fd1b19\",\"license\":\"GPL-2.0-or-later\",\"urls\":[\"bzz-raw://9624972f1c1a2465ff0ae8499cb5dbb80f97e3b669b1735fd84f7d587a9d8701\",\"dweb:/ipfs/QmXyDscxpznmg1AXhQap1ZWarv8XikPHpB2VBXWCJybuv4\"]},\"lib/v4-periphery/lib/v4-core/src/libraries/TickBitmap.sol\":{\"keccak256\":\"0x589cc6f6dd3a15b6f0b853c0936481f94159a1c8cd89d8132096ba53dc10d195\",\"license\":\"MIT\",\"urls\":[\"bzz-raw://66a0ee820dd4b231de97df125ae6ba6525f56a7076a5f7bccf7f14ef1123f23b\",\"dweb:/ipfs/QmR6nEkML7MsTKPFFwe8YARLubZ48N7E1x3VpbFpcjt2Za\"]},\"lib/v4-periphery/lib/v4-core/src/libraries/TickMath.sol\":{\"keccak256\":\"0x0fc7e6037541f82aea57c87bed4c397b4d7a564d792b850c516c33ca96a76fee\",\"license\":\"GPL-2.0-or-later\",\"urls\":[\"bzz-raw://6711ffba97c3bfbc34667621ea94fbdf3f99905dc0bfc8fb2dd9be158d837484\",\"dweb:/ipfs/QmSwM9FP42oV23ncuhNzeJJeumAeKrobxUhcgWNAFcqGsq\"]},\"lib/v4-periphery/lib/v4-core/src/libraries/UnsafeMath.sol\":{\"keccak256\":\"0x2a8cb75af05f8557b698b089a0ab19e24279fffb7e87cddf5794404c7eaf1c69\",\"license\":\"GPL-2.0-or-later\",\"urls\":[\"bzz-raw://85f68e6d614205f3f0f55f6ac41805e3a7d523d95bda1c443f2e75444443ffe3\",\"dweb:/ipfs/QmdCheMqxWNx4CbxG5MH9BUQeSMeeXF6DHkf7Aax4duRUe\"]},\"lib/v4-periphery/lib/v4-core/src/types/BalanceDelta.sol\":{\"keccak256\":\"0x9de918225b8c63a400c2ab6cf1fd28556a627f526e9254a73165c970b5366d76\",\"license\":\"MIT\",\"urls\":[\"bzz-raw://412d06a31961e198704dcd54a3a01c9ab72e5b5c3e8bd871a8a077e4c078a71b\",\"dweb:/ipfs/QmYFit1qnFhf9jmcTP2VjkH44Ea2ENgwsVmDuAf2UVeEnZ\"]},\"lib/v4-periphery/lib/v4-core/src/types/BeforeSwapDelta.sol\":{\"keccak256\":\"0xd31faab67e77831d68a64a82a6e1b0e79ca20f694cc21b7c18288c54ffee4ad3\",\"license\":\"MIT\",\"urls\":[\"bzz-raw://0f5aae3bdfc989bdc9f93835419f013dbac0cc4026bb5c86c84c7c69f1e66a2f\",\"dweb:/ipfs/QmSjL4d1dHo3w15trCDcawuSsiVNXGAUAQw477xQR3NVHV\"]},\"lib/v4-periphery/lib/v4-core/src/types/Currency.sol\":{\"keccak256\":\"0xb25dcf9675553b0dffbabe981ce420565accdf3d14ed80983c1a51d3f47b5a1f\",\"license\":\"MIT\",\"urls\":[\"bzz-raw://af8cad3298951b86c918bbf65417530a83f86cfbc4c9db4f1e61c7d53f5866a5\",\"dweb:/ipfs/Qme2ZDk1BRRjTLbLcMXaf2oS5JoQ3ArftB771B2r7Kntu3\"]},\"lib/v4-periphery/lib/v4-core/src/types/PoolId.sol\":{\"keccak256\":\"0xcf6a5090f89d80518af18561013f7d494b6d35ac2ad3fa34f04d138cb0a5ee41\",\"license\":\"MIT\",\"urls\":[\"bzz-raw://6dd98ef0562a116ee55cdd5ce4ca52d63433554ad5783c5cce915cf257d64411\",\"dweb:/ipfs/QmdpCVAQrJom1NwfeK1wJ8tSHiGoDriBt8DbwGeoYpVvJd\"]},\"lib/v4-periphery/lib/v4-core/src/types/PoolKey.sol\":{\"keccak256\":\"0xc51b7d99ae4067ff1697bacf6edb2d9722359c3e7780517b076eaefc786f5b8c\",\"license\":\"MIT\",\"urls\":[\"bzz-raw://8efbee019e87c1c8f4d7ab2de16cd63e55e45518e098e5339e0bb91e97cd98aa\",\"dweb:/ipfs/QmaNGMHWdUygghK1YWNV1g1Gwkj969whgqD8ZA8NsqxksJ\"]},\"lib/v4-periphery/lib/v4-core/src/types/Slot0.sol\":{\"keccak256\":\"0xc33c430fd7d292014e11a8c722e806a90b017a6e49bc3c701baa54068909a5c1\",\"license\":\"MIT\",\"urls\":[\"bzz-raw://34a1bd308ec0e083ef97ab8faef2dececbae7fc1598c372c7e57917311e93ef8\",\"dweb:/ipfs/QmcvjTuiZ4gB6AWu32JGw2pRnWifG6n4EeSFS6Bpvvyvgo\"]},\"src/generated/test_1.sol\":{\"keccak256\":\"0x8372c48f4c0abeeb70daae162c91e330d52b205b20e53c0e597b9ba64c8e9c7c\",\"license\":\"MIT\",\"urls\":[\"bzz-raw://c2b21a500b402b12b9d70934b0dd5ddfade327b184b6b998e2caca7ba41fcc28\",\"dweb:/ipfs/QmaqSWBwJPE3XyMN4eEg4nJs5U2v62iwEbJXAyy9k9Tjm9\"]}},\"version\":1}","metadata":{"compiler":{"version":"0.8.25+commit.b61c2a91"},"language":"Solidity","output":{"abi":[{"inputs":[{"internalType":"contract IPoolManager","name":"_poolManager","type":"address"}],"stateMutability":"nonpayable","type":"constructor"},{"inputs":[],"type":"error","name":"HookNotImplemented"},{"inputs":[],"type":"error","name":"InvalidPool"},{"inputs":[],"type":"error","name":"LockFailure"},{"inputs":[],"type":"error","name":"NotManager"},{"inputs":[],"type":"error","name":"NotSelf"},{"inputs":[{"internalType":"address","name":"","type":"address"},{"internalType":"struct PoolKey","name":"","type":"tuple","components":[{"internalType":"Currency","name":"currency0","type":"address"},{"internalType":"Currency","name":"currency1","type":"address"},{"internalType":"uint24","name":"fee","type":"uint24"},{"internalType":"int24","name":"tickSpacing","type":"int24"},{"internalType":"contract IHooks","name":"hooks","type":"address"}]},{"internalType":"struct IPoolManager.ModifyLiquidityParams","name":"","type":"tuple","components":[{"internalType":"int24","name":"tickLower","type":"int24"},{"internalType":"int24","name":"tickUpper","type":"int24"},{"internalType":"int256","name":"liquidityDelta","type":"int256"},{"internalType":"bytes32","name":"salt","type":"bytes32"}]},{"internalType":"BalanceDelta","name":"","type":"int256"},{"internalType":"bytes","name":"","type":"bytes"}],"stateMutability":"nonpayable","type":"function","name":"afterAddLiquidity","outputs":[{"internalType":"bytes4","name":"","type":"bytes4"},{"internalType":"BalanceDelta","name":"","type":"int256"}]},{"inputs":[{"internalType":"address","name":"","type":"address"},{"internalType":"struct PoolKey","name":"","type":"tuple","components":[{"internalType":"Currency","name":"currency0","type":"address"},{"internalType":"Currency","name":"currency1","type":"address"},{"internalType":"uint24","name":"fee","type":"uint24"},{"internalType":"int24","name":"tickSpacing","type":"int24"},{"internalType":"contract IHooks","name":"hooks","type":"address"}]},{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"bytes","name":"","type":"bytes"}],"stateMutability":"nonpayable","type":"function","name":"afterDonate","outputs":[{"internalType":"bytes4","name":"","type":"bytes4"}]},{"inputs":[{"internalType":"address","name":"","type":"address"},{"internalType":"struct PoolKey","name":"","type":"tuple","components":[{"internalType":"Currency","name":"currency0","type":"address"},{"internalType":"Currency","name":"currency1","type":"address"},{"internalType":"uint24","name":"fee","type":"uint24"},{"internalType":"int24","name":"tickSpacing","type":"int24"},{"internalType":"contract IHooks","name":"hooks","type":"address"}]},{"internalType":"uint160","name":"","type":"uint160"},{"internalType":"int24","name":"","type":"int24"},{"internalType":"bytes","name":"","type":"bytes"}],"stateMutability":"nonpayable","type":"function","name":"afterInitialize","outputs":[{"internalType":"bytes4","name":"","type":"bytes4"}]},{"inputs":[{"internalType":"address","name":"","type":"address"},{"internalType":"struct PoolKey","name":"","type":"tuple","components":[{"internalType":"Currency","name":"currency0","type":"address"},{"internalType":"Currency","name":"currency1","type":"address"},{"internalType":"uint24","name":"fee","type":"uint24"},{"internalType":"int24","name":"tickSpacing","type":"int24"},{"internalType":"contract IHooks","name":"hooks","type":"address"}]},{"internalType":"struct IPoolManager.ModifyLiquidityParams","name":"","type":"tuple","components":[{"internalType":"int24","name":"tickLower","type":"int24"},{"internalType":"int24","name":"tickUpper","type":"int24"},{"internalType":"int256","name":"liquidityDelta","type":"int256"},{"internalType":"bytes32","name":"salt","type":"bytes32"}]},{"internalType":"BalanceDelta","name":"","type":"int256"},{"internalType":"bytes","name":"","type":"bytes"}],"stateMutability":"nonpayable","type":"function","name":"afterRemoveLiquidity","outputs":[{"internalType":"bytes4","name":"","type":"bytes4"},{"internalType":"BalanceDelta","name":"","type":"int256"}]},{"inputs":[{"internalType":"address","name":"","type":"address"},{"internalType":"struct PoolKey","name":"","type":"tuple","components":[{"internalType":"Currency","name":"currency0","type":"address"},{"internalType":"Currency","name":"currency1","type":"address"},{"internalType":"uint24","name":"fee","type":"uint24"},{"internalType":"int24","name":"tickSpacing","type":"int24"},{"internalType":"contract IHooks","name":"hooks","type":"address"}]},{"internalType":"struct IPoolManager.SwapParams","name":"","type":"tuple","components":[{"internalType":"bool","name":"zeroForOne","type":"bool"},{"internalType":"int256","name":"amountSpecified","type":"int256"},{"internalType":"uint160","name":"sqrtPriceLimitX96","type":"uint160"}]},{"internalType":"BalanceDelta","name":"","type":"int256"},{"internalType":"bytes","name":"","type":"bytes"}],"stateMutability":"nonpayable","type":"function","name":"afterSwap","outputs":[{"internalType":"bytes4","name":"","type":"bytes4"},{"internalType":"int128","name":"","type":"int128"}]},{"inputs":[{"internalType":"address","name":"","type":"address"},{"internalType":"struct PoolKey","name":"","type":"tuple","components":[{"internalType":"Currency","name":"currency0","type":"address"},{"internalType":"Currency","name":"currency1","type":"address"},{"internalType":"uint24","name":"fee","type":"uint24"},{"internalType":"int24","name":"tickSpacing","type":"int24"},{"internalType":"contract IHooks","name":"hooks","type":"address"}]},{"internalType":"struct IPoolManager.ModifyLiquidityParams","name":"","type":"tuple","components":[{"internalType":"int24","name":"tickLower","type":"int24"},{"internalType":"int24","name":"tickUpper","type":"int24"},{"internalType":"int256","name":"liquidityDelta","type":"int256"},{"internalType":"bytes32","name":"salt","type":"bytes32"}]},{"internalType":"bytes","name":"","type":"bytes"}],"stateMutability":"nonpayable","type":"function","name":"beforeAddLiquidity","outputs":[{"internalType":"bytes4","name":"","type":"bytes4"}]},{"inputs":[{"internalType":"address","name":"","type":"address"},{"internalType":"struct PoolKey","name":"","type":"tuple","components":[{"internalType":"Currency","name":"currency0","type":"address"},{"internalType":"Currency","name":"currency1","type":"address"},{"internalType":"uint24","name":"fee","type":"uint24"},{"internalType":"int24","name":"tickSpacing","type":"int24"},{"internalType":"contract IHooks","name":"hooks","type":"address"}]},{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"bytes","name":"","type":"bytes"}],"stateMutability":"nonpayable","type":"function","name":"beforeDonate","outputs":[{"internalType":"bytes4","name":"","type":"bytes4"}]},{"inputs":[{"internalType":"address","name":"","type":"address"},{"internalType":"struct PoolKey","name":"","type":"tuple","components":[{"internalType":"Currency","name":"currency0","type":"address"},{"internalType":"Currency","name":"currency1","type":"address"},{"internalType":"uint24","name":"fee","type":"uint24"},{"internalType":"int24","name":"tickSpacing","type":"int24"},{"internalType":"contract IHooks","name":"hooks","type":"address"}]},{"internalType":"uint160","name":"","type":"uint160"},{"internalType":"bytes","name":"","type":"bytes"}],"stateMutability":"nonpayable","type":"function","name":"beforeInitialize","outputs":[{"internalType":"bytes4","name":"","type":"bytes4"}]},{"inputs":[{"internalType":"address","name":"","type":"address"},{"internalType":"struct PoolKey","name":"","type":"tuple","components":[{"internalType":"Currency","name":"currency0","type":"address"},{"internalType":"Currency","name":"currency1","type":"address"},{"internalType":"uint24","name":"fee","type":"uint24"},{"internalType":"int24","name":"tickSpacing","type":"int24"},{"internalType":"contract IHooks","name":"hooks","type":"address"}]},{"internalType":"struct IPoolManager.ModifyLiquidityParams","name":"","type":"tuple","components":[{"internalType":"int24","name":"tickLower","type":"int24"},{"internalType":"int24","name":"tickUpper","type":"int24"},{"internalType":"int256","name":"liquidityDelta","type":"int256"},{"internalType":"bytes32","name":"salt","type":"bytes32"}]},{"internalType":"bytes","name":"","type":"bytes"}],"stateMutability":"nonpayable","type":"function","name":"beforeRemoveLiquidity","outputs":[{"internalType":"bytes4","name":"","type":"bytes4"}]},{"inputs":[{"internalType":"address","name":"","type":"address"},{"internalType":"struct PoolKey","name":"key","type":"tuple","components":[{"internalType":"Currency","name":"currency0","type":"address"},{"internalType":"Currency","name":"currency1","type":"address"},{"internalType":"uint24","name":"fee","type":"uint24"},{"internalType":"int24","name":"tickSpacing","type":"int24"},{"internalType":"contract IHooks","name":"hooks","type":"address"}]},{"internalType":"struct IPoolManager.SwapParams","name":"","type":"tuple","components":[{"internalType":"bool","name":"zeroForOne","type":"bool"},{"internalType":"int256","name":"amountSpecified","type":"int256"},{"internalType":"uint160","name":"sqrtPriceLimitX96","type":"uint160"}]},{"internalType":"bytes","name":"","type":"bytes"}],"stateMutability":"nonpayable","type":"function","name":"beforeSwap","outputs":[{"internalType":"bytes4","name":"","type":"bytes4"},{"internalType":"BeforeSwapDelta","name":"","type":"int256"},{"internalType":"uint24","name":"","type":"uint24"}]},{"inputs":[],"stateMutability":"pure","type":"function","name":"getHookPermissions","outputs":[{"internalType":"struct Hooks.Permissions","name":"","type":"tuple","components":[{"internalType":"bool","name":"beforeInitialize","type":"bool"},{"internalType":"bool","name":"afterInitialize","type":"bool"},{"internalType":"bool","name":"beforeAddLiquidity","type":"bool"},{"internalType":"bool","name":"afterAddLiquidity","type":"bool"},{"internalType":"bool","name":"beforeRemoveLiquidity","type":"bool"},{"internalType":"bool","name":"afterRemoveLiquidity","type":"bool"},{"internalType":"bool","name":"beforeSwap","type":"bool"},{"internalType":"bool","name":"afterSwap","type":"bool"},{"internalType":"bool","name":"beforeDonate","type":"bool"},{"internalType":"bool","name":"afterDonate","type":"bool"},{"internalType":"bool","name":"beforeSwapReturnDelta","type":"bool"},{"internalType":"bool","name":"afterSwapReturnDelta","type":"bool"},{"internalType":"bool","name":"afterAddLiquidityReturnDelta","type":"bool"},{"internalType":"bool","name":"afterRemoveLiquidityReturnDelta","type":"bool"}]}]},{"inputs":[],"stateMutability":"view","type":"function","name":"manager","outputs":[{"internalType":"contract IPoolManager","name":"","type":"address"}]},{"inputs":[{"internalType":"PoolId","name":"","type":"bytes32"}],"stateMutability":"view","type":"function","name":"swapCount","outputs":[{"internalType":"uint256","name":"","type":"uint256"}]},{"inputs":[{"internalType":"bytes","name":"data","type":"bytes"}],"stateMutability":"nonpayable","type":"function","name":"unlockCallback","outputs":[{"internalType":"bytes","name":"","type":"bytes"}]}],"devdoc":{"kind":"dev","methods":{"unlockCallback(bytes)":{"details":"We force the onlyByManager modifier by exposing a virtual function after the onlyByManager check."}},"version":1},"userdoc":{"kind":"user","methods":{},"version":1}},"settings":{"remappings":["@axiom-crypto/v2-periphery/=lib/axiom-std/lib/axiom-v2-periphery/src/","@ensdomains/=lib/v4-periphery/lib/v4-core/node_modules/@ensdomains/","@openzeppelin/=lib/v4-periphery/lib/v4-core/lib/openzeppelin-contracts/","@uniswap/v4-core/=lib/v4-periphery/lib/v4-core/","axiom-std/=lib/axiom-std/src/","axiom-v2-periphery/=lib/axiom-std/lib/axiom-v2-periphery/","ds-test/=lib/v4-periphery/lib/forge-std/lib/ds-test/src/","erc4626-tests/=lib/v4-periphery/lib/openzeppelin-contracts/lib/erc4626-tests/","forge-gas-snapshot/=lib/v4-periphery/lib/forge-gas-snapshot/src/","forge-std/=lib/forge-std/src/","hardhat/=lib/v4-periphery/lib/v4-core/node_modules/hardhat/","openzeppelin-contracts/=lib/v4-periphery/lib/v4-core/lib/openzeppelin-contracts/","openzeppelin/=lib/v4-periphery/lib/openzeppelin-contracts/contracts/","solmate/=lib/v4-periphery/lib/solmate/src/","v4-core/=lib/v4-periphery/lib/v4-core/","v4-periphery/=lib/v4-periphery/contracts/"],"optimizer":{"enabled":true,"runs":800},"metadata":{"bytecodeHash":"ipfs"},"compilationTarget":{"src/generated/test_1.sol":"SimpleCounterHook"},"evmVersion":"cancun","libraries":{}},"sources":{"lib/v4-periphery/contracts/BaseHook.sol":{"keccak256":"0x799f21eb68a0f64398c6cb63f4911a06c471796d2bb340bb16943195a55c1085","urls":["bzz-raw://dcdd6f7d8a8c7d4b6f297ed2b81539d14fa84dc9fab09732c83f6a0c0dcf4651","dweb:/ipfs/QmQoXjxGmDgDfKNjdHBT4m6zNUGcB35CZQBrZK9VBJhydK"],"license":"UNLICENSED"},"lib/v4-periphery/contracts/base/ImmutableState.sol":{"keccak256":"0xef7c9afd38504de0b5c1f0489cebbf100d3c6f71f2035d1659c8ae90dafe4e99","urls":["bzz-raw://4f73fc15f80565debf95f184e6cbb4cbefe775eb6b615651b0e9dd161ce541b9","dweb:/ipfs/Qmdu4FpSNzujoafXEKeUZnArX4NUqzdRcrnM8bTAXBHToC"],"license":"GPL-2.0-or-later"},"lib/v4-periphery/contracts/base/SafeCallback.sol":{"keccak256":"0x2a73d4104f47cf6b81ecfe6713b2c59135ef1821dd13b242d26b3bd7239ee78f","urls":["bzz-raw://fe07ea01d76fe78f19def57a409960fc00c122715cba310665e31ffa052ed048","dweb:/ipfs/QmSpQTs3VUAQgfGuKpZ94gk2jQT86aWw27HAc1NCdTkMSX"],"license":"UNLICENSED"},"lib/v4-periphery/lib/v4-core/src/interfaces/IExtsload.sol":{"keccak256":"0x5457e68d4c21dd247bf5e3bea59df3372b197afb1856130b3512e101269742c9","urls":["bzz-raw://ec7ebfb1bea371ec76f2d8d9cdb5729176cd56432f20284042c8bd303c9b6221","dweb:/ipfs/QmaP2NLoyYmB1nxEU3RAEP7Q6AJBdLvjaGgfihLVXcuWgz"],"license":"MIT"},"lib/v4-periphery/lib/v4-core/src/interfaces/IExttload.sol":{"keccak256":"0xc70fcba06b34aa85a53476b251c60a72272ebf83784e39fd9f0a3a75977585a3","urls":["bzz-raw://b20481fe84db25f9bb896bb7afd1e6a6dfc70b6d4a67d9d1f282fb1214bd2f10","dweb:/ipfs/QmNhDbqgSdhx9Bbtifk5YX6Dpj8yKKKJf75y7JijYPDrF1"],"license":"MIT"},"lib/v4-periphery/lib/v4-core/src/interfaces/IHooks.sol":{"keccak256":"0x8751728dcb4923b86dbd7523e7a5a515177255bf64d6d95daaa00d510e97fb75","urls":["bzz-raw://92e9a4932961ec52fbca3fc3dd1704ac9c30541cfea7884c1d00a44632102bca","dweb:/ipfs/QmZWFtusFQLK9kXJLGWEMEHgV8LtvX9cKomqncF2mscJro"],"license":"MIT"},"lib/v4-periphery/lib/v4-core/src/interfaces/IPoolManager.sol":{"keccak256":"0xcfc4084f2519a4768a974a2b3a577866949674e76e10cc55f0e00ec6a910d38d","urls":["bzz-raw://769a40fded9d60d41a1443ad7a33b92972d7d0de8e93325ee90de6a35e2d85de","dweb:/ipfs/Qmbp5sPXTBDaRTJq5cggW7JpPGvNp3REkFUAS2dxh4xcwn"],"license":"MIT"},"lib/v4-periphery/lib/v4-core/src/interfaces/IProtocolFeeController.sol":{"keccak256":"0x0a48b2b660ef63aa520640bd7a80fdf7cf1fd19a268f155b029e5ca31b8ae4a6","urls":["bzz-raw://a2a74397cc05a7c32347c2bc63ab7dd4a7b436c7bb48b7e1910bc33a550f8116","dweb:/ipfs/QmXd91nmFT87pZc5DkZD4wQ6E2exLNAMkm2qCrCBN12de3"],"license":"MIT"},"lib/v4-periphery/lib/v4-core/src/interfaces/IProtocolFees.sol":{"keccak256":"0x78eb297b0e978d1b8217292a818d69ecfc403b4a3562d2eb9dfd7f1b781dc38b","urls":["bzz-raw://9abdc8e151e8bceb40d3667fc0345cd69b7379725f720490401ad3775527bf68","dweb:/ipfs/QmNoHFjChTCoC4a2TvomtGssddq7j2Rga4mrSwFm1VQ6My"],"license":"MIT"},"lib/v4-periphery/lib/v4-core/src/interfaces/callback/IUnlockCallback.sol":{"keccak256":"0xe413ff595dadedfeff672247031151c309f15a104b7ed03a51400e50c337d373","urls":["bzz-raw://1558caec33b3af1e3b8080973cd1f7f2e4f1483b06c31dbb84cdc2414b1f5a1e","dweb:/ipfs/QmYgQXpMTvb3ZJex1s61QKZQ9WRRqFg6HRiYndgPh7yKT1"],"license":"MIT"},"lib/v4-periphery/lib/v4-core/src/interfaces/external/IERC20Minimal.sol":{"keccak256":"0x8dc5ee72426906aeb12f4ae6a2426df059491739bd5cbdb8ea4a3e26970436c6","urls":["bzz-raw://f7e30dbbc972cb78d169c8845d6210d27490c8534e68a20d4f296b11ec727b92","dweb:/ipfs/QmXBvaWaAnPW25YGVm2sMbf5mwrzLapUcqPnrNiiGGXxmA"],"license":"GPL-2.0-or-later"},"lib/v4-periphery/lib/v4-core/src/interfaces/external/IERC6909Claims.sol":{"keccak256":"0x1d53aa307e997fd73e1a31abe1052aad574994627958399375f760b8a750afbc","urls":["bzz-raw://c308d1aba00f3f834357cea75e03a64eba3da425eb31ed6bd4a9e56ada606b79","dweb:/ipfs/Qmb8fuGxpw1JRRFdXUF9U2Una6msqYFTXMvw6ULPHpJ4ow"],"license":"MIT"},"lib/v4-periphery/lib/v4-core/src/libraries/BitMath.sol":{"keccak256":"0x652cd36970d953b3067c766b3b0c0063d4c403eba7352df09ec10dcc868910d9","urls":["bzz-raw://e357b092171edb755c8514b4b4ae758a8b43458c7a23409ebf5afd325fefa2be","dweb:/ipfs/QmWKRWT2z15cVbVVDkjqdVnpf8Ck1jytcXbA7zCUevp7b6"],"license":"GPL-2.0-or-later"},"lib/v4-periphery/lib/v4-core/src/libraries/CustomRevert.sol":{"keccak256":"0xd768d9e1ebdaa14b723a1bd4faf55c37104fa28c00bb8ee399dc8d55f0da480c","urls":["bzz-raw://683906f0281596df504b8b711ca8d1d48f65e03f4dd649802118e03469ebe421","dweb:/ipfs/QmXcpr5RNefsAd33hmUDjbRVy2mRRrQtPHbDE3kYnLx4KR"],"license":"MIT"},"lib/v4-periphery/lib/v4-core/src/libraries/FixedPoint128.sol":{"keccak256":"0x1f015a7a107f4c471985a958f1bfbc1710948f9bcd3786a02310bc0ad5080c69","urls":["bzz-raw://a1cacac08675284f319d151f6bc5d1c66cf1d54c5dad1c620a2e1e1336cbb93a","dweb:/ipfs/QmPf8vQ3mBqniKHwvS1DDEFrV5dJfZSL6mMi9JVy6omQaz"],"license":"GPL-2.0-or-later"},"lib/v4-periphery/lib/v4-core/src/libraries/FixedPoint96.sol":{"keccak256":"0x7731aeeba692b0ca5dac8f27803ec9e5ce98795a554c09ae05c220861bd0314e","urls":["bzz-raw://963a8155e442d31c4541c387d333db0c02941ae3517e9173ad30ebcb4f6a14e9","dweb:/ipfs/QmevAr8oG4WSoi5gBqQ53VzBuFbDKoZHfJwjuTmVhzm8Tc"],"license":"GPL-2.0-or-later"},"lib/v4-periphery/lib/v4-core/src/libraries/FullMath.sol":{"keccak256":"0x480e1e2741db0d1d5843c81d25576f182f64db9b92788ccbfc321a05d726f5e3","urls":["bzz-raw://f5c9606989f6487189a70e383f88ec0b203f8232c6e901e51e421113008959fd","dweb:/ipfs/QmeHAmfavKjLtcd1J53Bs9WbknH3jLZfmU9NeaUZCHCJwY"],"license":"MIT"},"lib/v4-periphery/lib/v4-core/src/libraries/Hooks.sol":{"keccak256":"0x036b3d7f46a6bc2f5d4d563a926baad015cb76f041b743af1a343b3f418a57cf","urls":["bzz-raw://f1e555dd1854d4c390b750ae23fc36818c0e2237f562fdcfd8404241825234b8","dweb:/ipfs/QmcV3zQrydELX8G3nJRVLnA9SG5jar1uJejww9jbvuQ3cm"],"license":"MIT"},"lib/v4-periphery/lib/v4-core/src/libraries/LPFeeLibrary.sol":{"keccak256":"0x33f69b500fd414cdb9856d9e623d085f8a15a8d3a9570ac6339514c800d077bf","urls":["bzz-raw://c45ed697cee974ecd23cdd1995fb837cf6e8a9b18a57c1d5edfbd0e60a61e68c","dweb:/ipfs/QmTNrppHysZU3WLt4TtFJmQ9CRbVUbk2Nfu6qoKiVCom2P"],"license":"GPL-2.0-or-later"},"lib/v4-periphery/lib/v4-core/src/libraries/LiquidityMath.sol":{"keccak256":"0xf60e2d395e715919a6cb0111dde08055a5983dce63dfdeef345b33f9a83a63fe","urls":["bzz-raw://fdbcf1d6df9c0ca34e133ca7fd90f6aba7bed873d397a07cd8c4e37bd69c23d8","dweb:/ipfs/QmPMLysVr1yn5ZYMbC6g1MHL8RC4RCmk6omZzTePHJ9Kte"],"license":"GPL-2.0-or-later"},"lib/v4-periphery/lib/v4-core/src/libraries/ParseBytes.sol":{"keccak256":"0x6ce53c8138982fed86ce0a0f53c6859e1102c2b3105201320364d1f2f5f5d555","urls":["bzz-raw://b5891c8a94fad51a98764adea8940a8f13275fe0732d58bb1480c2405846a9e6","dweb:/ipfs/QmPB31BvQ8DgaUjj1iSUdJxLMtMJhfMSEvnNKiQs18kTdq"],"license":"GPL-2.0-or-later"},"lib/v4-periphery/lib/v4-core/src/libraries/Pool.sol":{"keccak256":"0x5872d9930358dba79a19efacdf3993b541be408fbd8c301c23f0e4b344d2cec2","urls":["bzz-raw://db6532f4eb1052a9df508d90ff070f67581c83be13e2e4a03062392ff22559a1","dweb:/ipfs/QmSJX5Xzh38Zq1pGVmmS2239LocRdeYMmxi7TiEdeLzpUo"],"license":"BUSL-1.1"},"lib/v4-periphery/lib/v4-core/src/libraries/Position.sol":{"keccak256":"0x3f99796bda1592c6df7140c7f6341bedda3458f785f86455b8e147bd6fafeafa","urls":["bzz-raw://5c9c2aebfb7dee1971168c01cb31704dbb5645249cbd185353f9f04283ddd797","dweb:/ipfs/QmQe9hngtfaHpEcroUGxWqLp6ceWaD5rHv1FffqK6XJpj3"],"license":"BUSL-1.1"},"lib/v4-periphery/lib/v4-core/src/libraries/ProtocolFeeLibrary.sol":{"keccak256":"0xfe57793436b42e73edaf9281dbf170a03169e17123916dcaedc6bf6c63ed85cd","urls":["bzz-raw://be1430384e13155a63ae88732b6f562fe482b3d182bba352f5ae2d17d2edd382","dweb:/ipfs/QmeKZZNByL2Z6f1mJYZeztjkFKYddYpTQKkPuMMj9M67e2"],"license":"GPL-2.0-or-later"},"lib/v4-periphery/lib/v4-core/src/libraries/SafeCast.sol":{"keccak256":"0x3dbf539c83fbda8bf5373f6c66b720ee4df1725e7e56b478bd938ea6cea4faf1","urls":["bzz-raw://b18b54bc7503a5fcd65cf7473778e17de582b8ad994e620ec1101791caf96eac","dweb:/ipfs/QmVyh8EbAutw3F8vBPKBjLgoNJszqqb58zRzjVG64mydk6"],"license":"GPL-2.0-or-later"},"lib/v4-periphery/lib/v4-core/src/libraries/SqrtPriceMath.sol":{"keccak256":"0x7117071dc1c13358a28d1a8cdef483f62549e58c858fcdf734ddcb8545d40eae","urls":["bzz-raw://c8b93a915cc7fd9cd1e850555a0bd757b8fc33b99dbd39b2ab87930d72392620","dweb:/ipfs/QmcMHNmqotZM3iFRSHDqwvq7cDtk8DHuQLtNgu93PPUvgE"],"license":"GPL-2.0-or-later"},"lib/v4-periphery/lib/v4-core/src/libraries/SwapMath.sol":{"keccak256":"0xf0fc6e250b4f3d77d91ca2cb7ad26d0c4753656a5f567db53a602fbbe7fd1b19","urls":["bzz-raw://9624972f1c1a2465ff0ae8499cb5dbb80f97e3b669b1735fd84f7d587a9d8701","dweb:/ipfs/QmXyDscxpznmg1AXhQap1ZWarv8XikPHpB2VBXWCJybuv4"],"license":"GPL-2.0-or-later"},"lib/v4-periphery/lib/v4-core/src/libraries/TickBitmap.sol":{"keccak256":"0x589cc6f6dd3a15b6f0b853c0936481f94159a1c8cd89d8132096ba53dc10d195","urls":["bzz-raw://66a0ee820dd4b231de97df125ae6ba6525f56a7076a5f7bccf7f14ef1123f23b","dweb:/ipfs/QmR6nEkML7MsTKPFFwe8YARLubZ48N7E1x3VpbFpcjt2Za"],"license":"MIT"},"lib/v4-periphery/lib/v4-core/src/libraries/TickMath.sol":{"keccak256":"0x0fc7e6037541f82aea57c87bed4c397b4d7a564d792b850c516c33ca96a76fee","urls":["bzz-raw://6711ffba97c3bfbc34667621ea94fbdf3f99905dc0bfc8fb2dd9be158d837484","dweb:/ipfs/QmSwM9FP42oV23ncuhNzeJJeumAeKrobxUhcgWNAFcqGsq"],"license":"GPL-2.0-or-later"},"lib/v4-periphery/lib/v4-core/src/libraries/UnsafeMath.sol":{"keccak256":"0x2a8cb75af05f8557b698b089a0ab19e24279fffb7e87cddf5794404c7eaf1c69","urls":["bzz-raw://85f68e6d614205f3f0f55f6ac41805e3a7d523d95bda1c443f2e75444443ffe3","dweb:/ipfs/QmdCheMqxWNx4CbxG5MH9BUQeSMeeXF6DHkf7Aax4duRUe"],"license":"GPL-2.0-or-later"},"lib/v4-periphery/lib/v4-core/src/types/BalanceDelta.sol":{"keccak256":"0x9de918225b8c63a400c2ab6cf1fd28556a627f526e9254a73165c970b5366d76","urls":["bzz-raw://412d06a31961e198704dcd54a3a01c9ab72e5b5c3e8bd871a8a077e4c078a71b","dweb:/ipfs/QmYFit1qnFhf9jmcTP2VjkH44Ea2ENgwsVmDuAf2UVeEnZ"],"license":"MIT"},"lib/v4-periphery/lib/v4-core/src/types/BeforeSwapDelta.sol":{"keccak256":"0xd31faab67e77831d68a64a82a6e1b0e79ca20f694cc21b7c18288c54ffee4ad3","urls":["bzz-raw://0f5aae3bdfc989bdc9f93835419f013dbac0cc4026bb5c86c84c7c69f1e66a2f","dweb:/ipfs/QmSjL4d1dHo3w15trCDcawuSsiVNXGAUAQw477xQR3NVHV"],"license":"MIT"},"lib/v4-periphery/lib/v4-core/src/types/Currency.sol":{"keccak256":"0xb25dcf9675553b0dffbabe981ce420565accdf3d14ed80983c1a51d3f47b5a1f","urls":["bzz-raw://af8cad3298951b86c918bbf65417530a83f86cfbc4c9db4f1e61c7d53f5866a5","dweb:/ipfs/Qme2ZDk1BRRjTLbLcMXaf2oS5JoQ3ArftB771B2r7Kntu3"],"license":"MIT"},"lib/v4-periphery/lib/v4-core/src/types/PoolId.sol":{"keccak256":"0xcf6a5090f89d80518af18561013f7d494b6d35ac2ad3fa34f04d138cb0a5ee41","urls":["bzz-raw://6dd98ef0562a116ee55cdd5ce4ca52d63433554ad5783c5cce915cf257d64411","dweb:/ipfs/QmdpCVAQrJom1NwfeK1wJ8tSHiGoDriBt8DbwGeoYpVvJd"],"license":"MIT"},"lib/v4-periphery/lib/v4-core/src/types/PoolKey.sol":{"keccak256":"0xc51b7d99ae4067ff1697bacf6edb2d9722359c3e7780517b076eaefc786f5b8c","urls":["bzz-raw://8efbee019e87c1c8f4d7ab2de16cd63e55e45518e098e5339e0bb91e97cd98aa","dweb:/ipfs/QmaNGMHWdUygghK1YWNV1g1Gwkj969whgqD8ZA8NsqxksJ"],"license":"MIT"},"lib/v4-periphery/lib/v4-core/src/types/Slot0.sol":{"keccak256":"0xc33c430fd7d292014e11a8c722e806a90b017a6e49bc3c701baa54068909a5c1","urls":["bzz-raw://34a1bd308ec0e083ef97ab8faef2dececbae7fc1598c372c7e57917311e93ef8","dweb:/ipfs/QmcvjTuiZ4gB6AWu32JGw2pRnWifG6n4EeSFS6Bpvvyvgo"],"license":"MIT"},"src/generated/test_1.sol":{"keccak256":"0x8372c48f4c0abeeb70daae162c91e330d52b205b20e53c0e597b9ba64c8e9c7c","urls":["bzz-raw://c2b21a500b402b12b9d70934b0dd5ddfade327b184b6b998e2caca7ba41fcc28","dweb:/ipfs/QmaqSWBwJPE3XyMN4eEg4nJs5U2v62iwEbJXAyy9k9Tjm9"],"license":"MIT"}},"version":1},"id":88}

In [12]:
a["abi"]

[{'type': 'constructor',
  'inputs': [{'name': '_poolManager',
    'type': 'address',
    'internalType': 'contract IPoolManager'}],
  'stateMutability': 'nonpayable'},
 {'type': 'function',
  'name': 'afterAddLiquidity',
  'inputs': [{'name': '', 'type': 'address', 'internalType': 'address'},
   {'name': '',
    'type': 'tuple',
    'internalType': 'struct PoolKey',
    'components': [{'name': 'currency0',
      'type': 'address',
      'internalType': 'Currency'},
     {'name': 'currency1', 'type': 'address', 'internalType': 'Currency'},
     {'name': 'fee', 'type': 'uint24', 'internalType': 'uint24'},
     {'name': 'tickSpacing', 'type': 'int24', 'internalType': 'int24'},
     {'name': 'hooks', 'type': 'address', 'internalType': 'contract IHooks'}]},
   {'name': '',
    'type': 'tuple',
    'internalType': 'struct IPoolManager.ModifyLiquidityParams',
    'components': [{'name': 'tickLower',
      'type': 'int24',
      'internalType': 'int24'},
     {'name': 'tickUpper', 'type': 'in

In [13]:
bytecode = a["bytecode"]["object"]

In [14]:
aa = """// SPDX-License-Identifier: MIT\npragma solidity ^0.8.24;\n\nimport {BaseHook} from \"v4-periphery/BaseHook.sol\";\nimport {Hooks} from \"v4-core/src/libraries/Hooks.sol\";\nimport {IPoolManager} from \"v4-core/src/interfaces/IPoolManager.sol\";\nimport {PoolKey} from \"v4-core/src/types/PoolKey.sol\";\nimport {PoolId, PoolIdLibrary} from \"v4-core/src/types/PoolId.sol\";\nimport {BalanceDelta} from \"v4-core/src/types/BalanceDelta.sol\";\nimport {BeforeSwapDelta, BeforeSwapDeltaLibrary} from \"v4-core/src/types/BeforeSwapDelta.sol\";\n\ncontract SimpleSwapCounter is BaseHook {\n    using PoolIdLibrary for PoolKey;\n\n    // Counter to keep track of swaps per pool\n    mapping(PoolId => uint256) public swapCount;\n\n    constructor(IPoolManager _poolManager) BaseHook(_poolManager) {}\n\n    function getHookPermissions() public pure override returns (Hooks.Permissions memory) {\n        return Hooks.Permissions({\n            beforeInitialize: false,\n            afterInitialize: false,\n            beforeAddLiquidity: false,\n            afterAddLiquidity: false,\n            beforeRemoveLiquidity: false,\n            afterRemoveLiquidity: false,\n            beforeSwap: false,\n            afterSwap: true,\n            beforeDonate: false,\n            afterDonate: false,\n            beforeSwapReturnDelta: false,\n            afterSwapReturnDelta: false,\n            afterAddLiquidityReturnDelta: false,\n            afterRemoveLiquidityReturnDelta: false\n        });\n    }\n\n    // Function to increment the swap counter after each swap\n    function afterSwap(address, PoolKey calldata key, IPoolManager.SwapParams calldata, BalanceDelta, bytes calldata)\n        external\n        override\n        returns (bytes4, int128)\n    {\n        swapCount[key.toId()]++;\n        return (BaseHook.afterSwap.selector, 0);\n    }\n}\n"""

In [15]:
render_markdown(aa)

// SPDX-License-Identifier: MIT
pragma solidity ^0.8.24;

import {BaseHook} from "v4-periphery/BaseHook.sol";
import {Hooks} from "v4-core/src/libraries/Hooks.sol";
import {IPoolManager} from "v4-core/src/interfaces/IPoolManager.sol";
import {PoolKey} from "v4-core/src/types/PoolKey.sol";
import {PoolId, PoolIdLibrary} from "v4-core/src/types/PoolId.sol";
import {BalanceDelta} from "v4-core/src/types/BalanceDelta.sol";
import {BeforeSwapDelta, BeforeSwapDeltaLibrary} from "v4-core/src/types/BeforeSwapDelta.sol";

contract SimpleSwapCounter is BaseHook {
    using PoolIdLibrary for PoolKey;

    // Counter to keep track of swaps per pool
    mapping(PoolId => uint256) public swapCount;

    constructor(IPoolManager _poolManager) BaseHook(_poolManager) {}

    function getHookPermissions() public pure override returns (Hooks.Permissions memory) {
        return Hooks.Permissions({
            beforeInitialize: false,
            afterInitialize: false,
            beforeAddLiquidity: false,
            afterAddLiquidity: false,
            beforeRemoveLiquidity: false,
            afterRemoveLiquidity: false,
            beforeSwap: false,
            afterSwap: true,
            beforeDonate: false,
            afterDonate: false,
            beforeSwapReturnDelta: false,
            afterSwapReturnDelta: false,
            afterAddLiquidityReturnDelta: false,
            afterRemoveLiquidityReturnDelta: false
        });
    }

    // Function to increment the swap counter after each swap
    function afterSwap(address, PoolKey calldata key, IPoolManager.SwapParams calldata, BalanceDelta, bytes calldata)
        external
        override
        returns (bytes4, int128)
    {
        swapCount[key.toId()]++;
        return (BaseHook.afterSwap.selector, 0);
    }
}


In [17]:
from web3 import Web3
from eth_utils import keccak, to_bytes
from eth_account import Account

# Mask to slice out the top 10 bits of the address
FLAG_MASK = 0x3FF << 146

# Maximum number of iterations to find a salt, avoid infinite loops
MAX_LOOP = 100000

def find(deployer: str, flags: int, seed: int, creation_code: bytes, constructor_args: bytes):
    """
    Find a salt that produces a hook address with the desired `flags`

    :param deployer: The address that will deploy the hook.
                     In `forge test`, this will be the test contract `address(this)` or the pranking address
                     In `forge script`, this should be `0x4e59b44847b379578588920cA78FbF26c0B4956C` (CREATE2 Deployer Proxy)
    :param flags: The desired flags for the hook address
    :param seed: Use 0 for as a default. An optional starting salt when linearly searching for a salt
                 Useful for finding salts for multiple hooks with the same flags
    :param creation_code: The creation code of a hook contract. Example: `type(Counter).creationCode`
    :param constructor_args: The encoded constructor arguments of a hook contract. Example: `abi.encode(address(manager))`
    :return: A tuple containing the hook address and the salt that was found.
             The salt can be used in `new Hook{salt: salt}(<constructor arguments>)`
    """
    creation_code_with_args = creation_code + constructor_args

    for salt in range(seed, MAX_LOOP):
        hook_address = compute_address(deployer, salt, creation_code_with_args)
        #print(ethereum_address_to_binary_and_top_10(hook_address)[1], flags)
        if ethereum_address_to_binary_and_least_14_bits(hook_address)[1] == flags:
            return hook_address, Web3.to_hex(salt)

    raise Exception('HookMiner: could not find salt')

def compute_address(deployer: str, salt: int, creation_code: bytes) -> str:
    """
    Precompute a contract address deployed via CREATE2

    :param deployer: The address that will deploy the hook
                     In `forge test`, this will be the test contract `address(this)` or the pranking address
                     In `forge script`, this should be `0x4e59b44847b379578588920cA78FbF26c0B4956C` (CREATE2 Deployer Proxy)
    :param salt: The salt used to deploy the hook
    :param creation_code: The creation code of a hook contract
    :return: The computed address of the hook contract
    """
    deployer_bytes = to_bytes(hexstr=deployer)
    salt_bytes = Web3.to_bytes(salt)
    creation_code_hash = keccak(creation_code)
    
    return Web3.to_checksum_address(Web3.solidity_keccak(
        ['bytes1', 'address', 'bytes32', 'bytes32'],
        ['0xff', deployer, Web3.to_hex(salt_bytes), Web3.to_hex(creation_code_hash)]
    )[12:].hex())

def hex_to_bytes(hex_str: str) -> bytes:
    """
    Convert a hex string to bytes.
    
    :param hex_str: The hex string to convert
    :return: The resulting bytes
    """
    return bytes.fromhex(hex_str[2:])  # Remove the '0x' prefix before converting


# Example usage
deployer = '0x4e59b44847b379578588920cA78FbF26c0B4956C'
flags = 5435
seed = 0
creation_code = hex_to_bytes(bytecode)
constructor_args = hex_to_bytes(deployer)

hook_address, salt = find(deployer, flags, seed, creation_code, constructor_args)
print(f'Found hook address: {hook_address} with salt: {salt}')

NameError: name 'ethereum_address_to_binary_and_least_14_bits' is not defined

In [373]:
# Define the flags with uint160 constraint
BEFORE_INITIALIZE_FLAG = 1 << 13
AFTER_INITIALIZE_FLAG = 1 << 12

BEFORE_ADD_LIQUIDITY_FLAG = 1 << 11
AFTER_ADD_LIQUIDITY_FLAG = 1 << 10

BEFORE_REMOVE_LIQUIDITY_FLAG = 1 << 9
AFTER_REMOVE_LIQUIDITY_FLAG = 1 << 8

BEFORE_SWAP_FLAG = 1 << 7
AFTER_SWAP_FLAG = 1 << 6

BEFORE_DONATE_FLAG = 1 << 5
AFTER_DONATE_FLAG = 1 << 4

BEFORE_SWAP_RETURNS_DELTA_FLAG = 1 << 3
AFTER_SWAP_RETURNS_DELTA_FLAG = 1 << 2
AFTER_ADD_LIQUIDITY_RETURNS_DELTA_FLAG = 1 << 1
AFTER_REMOVE_LIQUIDITY_RETURNS_DELTA_FLAG = 1 << 0

FLAGS = {
    'BEFORE_INITIALIZE_FLAG': BEFORE_INITIALIZE_FLAG,
    'AFTER_INITIALIZE_FLAG': AFTER_INITIALIZE_FLAG,
    'BEFORE_ADD_LIQUIDITY_FLAG': BEFORE_ADD_LIQUIDITY_FLAG,
    'AFTER_ADD_LIQUIDITY_FLAG': AFTER_ADD_LIQUIDITY_FLAG,
    'BEFORE_REMOVE_LIQUIDITY_FLAG': BEFORE_REMOVE_LIQUIDITY_FLAG,
    'AFTER_REMOVE_LIQUIDITY_FLAG': AFTER_REMOVE_LIQUIDITY_FLAG,
    'BEFORE_SWAP_FLAG': BEFORE_SWAP_FLAG,
    'AFTER_SWAP_FLAG': AFTER_SWAP_FLAG,
    'BEFORE_DONATE_FLAG': BEFORE_DONATE_FLAG,
    'AFTER_DONATE_FLAG': AFTER_DONATE_FLAG,
    'BEFORE_SWAP_RETURNS_DELTA_FLAG': BEFORE_SWAP_RETURNS_DELTA_FLAG,
    'AFTER_SWAP_RETURNS_DELTA_FLAG': AFTER_SWAP_RETURNS_DELTA_FLAG,
    'AFTER_ADD_LIQUIDITY_RETURNS_DELTA_FLAG': AFTER_ADD_LIQUIDITY_RETURNS_DELTA_FLAG,
    'AFTER_REMOVE_LIQUIDITY_RETURNS_DELTA_FLAG': AFTER_REMOVE_LIQUIDITY_RETURNS_DELTA_FLAG,
}


In [374]:
def calculate_flags(*flag_names):
    """
    Calculate the combined uint160 value for the given flags.
    
    :param flag_names: The names of the flags to combine
    :return: The combined uint160 value
    """
    combined_flags = 0
    for name in flag_names:
        if name in FLAGS:
            combined_flags |= FLAGS[name]
        else:
            raise ValueError(f"Flag '{name}' is not defined.")
    return combined_flags

# Example usage
combined_flags = calculate_flags('AFTER_ADD_LIQUIDITY_FLAG', 'AFTER_SWAP_FLAG')
print(f"Combined Flags: {combined_flags}, Binary: {combined_flags:016b}")


Combined Flags: 1088, Binary: 0000010001000000


In [292]:
def binary_to_int(binary_str):
    """
    Convert a binary string to an integer.
    
    :param binary_str: The binary string to convert
    :return: The integer representation of the binary string
    """
    return int(binary_str, 2)

In [385]:
def int_to_binary(value, bit_length=160):
    """
    Convert an integer to its binary representation as a string, padded to the specified bit length.
    
    :param value: The integer to convert
    :param bit_length: The desired length of the binary string
    :return: The binary representation of the integer as a string, padded to the specified bit length
    """
    if value < 0:
        raise ValueError("Value cannot be negative")
    # Convert to binary and remove the '0b' prefix
    binary_representation = bin(value)[2:]
    
    # Pad the binary representation to ensure it is the specified bit length
    padded_binary = binary_representation.zfill(bit_length)
    
    return padded_binary

def ethereum_address_to_binary_and_least_14_bits(address):
    """
    Convert an Ethereum address to its 160-bit binary representation.
    
    :param address: The Ethereum address in hexadecimal format (with "0x" prefix)
    :return: The 160-bit binary representation of the address as a string
    """
    if not address.startswith('0x'):
        raise ValueError("Ethereum address must start with '0x'")
    
    # Remove the '0x' prefix
    hex_str = address[2:]
    
    # Convert the hexadecimal string to an integer
    address_int = int(hex_str, 16)
    
    # Convert the integer to its binary representation and pad to 160 bits
    binary_representation = bin(address_int)[2:].zfill(160)

    least_14_bits = address_int & 0x3FFF
    
    return binary_representation, least_14_bits

# Example usage
ethereum_address = "0x4e59b44847b379578588920cA78FbF26c0B4956C"


In [382]:
ethereum_address_to_binary_and_least_14_bits(hook_address)

('1111111110100101001101111111000000111011011101111100100011110110101110011100110010100010111000101101100100100010010010111101111111101110010110001100110011010111',
 3287)

In [389]:
import re

# Define the flags with uint160 constraint
BEFORE_INITIALIZE_FLAG = 1 << 13
AFTER_INITIALIZE_FLAG = 1 << 12

BEFORE_ADD_LIQUIDITY_FLAG = 1 << 11
AFTER_ADD_LIQUIDITY_FLAG = 1 << 10

BEFORE_REMOVE_LIQUIDITY_FLAG = 1 << 9
AFTER_REMOVE_LIQUIDITY_FLAG = 1 << 8

BEFORE_SWAP_FLAG = 1 << 7
AFTER_SWAP_FLAG = 1 << 6

BEFORE_DONATE_FLAG = 1 << 5
AFTER_DONATE_FLAG = 1 << 4

BEFORE_SWAP_RETURNS_DELTA_FLAG = 1 << 3
AFTER_SWAP_RETURNS_DELTA_FLAG = 1 << 2
AFTER_ADD_LIQUIDITY_RETURNS_DELTA_FLAG = 1 << 1
AFTER_REMOVE_LIQUIDITY_RETURNS_DELTA_FLAG = 1 << 0

# Mapping the flag names to their corresponding values
FLAGS = {
    'beforeInitialize': BEFORE_INITIALIZE_FLAG,
    'afterInitialize': AFTER_INITIALIZE_FLAG,
    'beforeAddLiquidity': BEFORE_ADD_LIQUIDITY_FLAG,
    'afterAddLiquidity': AFTER_ADD_LIQUIDITY_FLAG,
    'beforeRemoveLiquidity': BEFORE_REMOVE_LIQUIDITY_FLAG,
    'afterRemoveLiquidity': AFTER_REMOVE_LIQUIDITY_FLAG,
    'beforeSwap': BEFORE_SWAP_FLAG,
    'afterSwap': AFTER_SWAP_FLAG,
    'beforeDonate': BEFORE_DONATE_FLAG,
    'afterDonate': AFTER_DONATE_FLAG,
    'beforeSwapReturnDelta': BEFORE_SWAP_RETURNS_DELTA_FLAG,
    'afterSwapReturnDelta': AFTER_SWAP_RETURNS_DELTA_FLAG,
    'afterAddLiquidityReturnDelta': AFTER_ADD_LIQUIDITY_RETURNS_DELTA_FLAG,
    'afterRemoveLiquidityReturnDelta': AFTER_REMOVE_LIQUIDITY_RETURNS_DELTA_FLAG,
}

def extract_flags_from_code(code):
    """
    Extract flag values from Solidity code using regex.
    
    :param code: The Solidity code as a string
    :return: A dictionary representing the state of each flag (true or false)
    """
    flag_states = {}
    pattern = re.compile(r'(\w+):\s*(true|false)', re.IGNORECASE)
    matches = pattern.findall(code)
    for flag, state in matches:
        flag_states[flag] = state.lower() == 'true'
    return flag_states

def calculate_flags(flag_states):
    """
    Calculate the combined flags based on the provided flag states.
    
    :param flag_states: A dictionary representing the state of each flag (true or false)
    :return: The combined flags as an integer
    """
    combined_flags = 0
    for flag_name, flag_value in FLAGS.items():
        if flag_states.get(flag_name, False):
            combined_flags |= flag_value
    return combined_flags

# Extract the flag states from the Solidity code
flag_states = extract_flags_from_code(solidity_code)
combined_flags = calculate_flags(flag_states)


Flag States: {'beforeInitialize': False, 'afterInitialize': False, 'beforeAddLiquidity': False, 'beforeRemoveLiquidity': False, 'afterAddLiquidity': False, 'afterRemoveLiquidity': False, 'beforeSwap': True, 'afterSwap': False, 'beforeDonate': False, 'afterDonate': False, 'beforeSwapReturnDelta': False, 'afterSwapReturnDelta': False, 'afterAddLiquidityReturnDelta': False, 'afterRemoveLiquidityReturnDelta': False}
Combined Flags (Decimal): 128
Combined Flags (Binary): 0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000010000000


In [390]:
combined_flags

128

In [395]:
def remove_last_brace(text):
    """
    Remove the last occurrence of '}' in the text.
    
    :param text: The input string
    :return: A string with the last '}' removed
    """
    last_brace_index = text.rfind('}')
    
    if last_brace_index != -1:
        # Return the string with the last '}' removed
        return text[:last_brace_index] + text[last_brace_index + 1:]
    else:
        # Return the original text if no '}' is found
        return text

In [396]:
render_markdown(remove_last_brace(answer))

// SPDX-License-Identifier: UNLICENSED
pragma solidity >=0.8.19;

import {Ownable} from "@openzeppelin/contracts/access/Ownable.sol";
import {IPoolManager} from "v4-core/src/interfaces/IPoolManager.sol";
import {Hooks} from "v4-core/src/libraries/Hooks.sol";
import {BaseHook} from "v4-periphery/BaseHook.sol";
import {PoolKey} from "v4-core/src/types/PoolKey.sol";
import {BeforeSwapDelta, BeforeSwapDeltaLibrary} from "v4-core/src/types/BeforeSwapDelta.sol";

/**
 * @title An interface for checking whether an address has a valid kycNFT token
 */
interface IKycValidity {
    /// @dev Check whether a given address has a valid kycNFT token
    /// @param _addr Address to check for tokens
    /// @return valid Whether the address has a valid token
    function hasValidToken(address _addr) external view returns (bool valid);
}

/**
 * Only KYC'ed people can trade on the V4 hook'ed pool.
 * Caveat: Relies on external oracle for info on an address's KYC status.
 */
contract KYCSwaps is BaseHook, Ownable {
    IKycValidity public kycValidity;
    address private _preKycValidity;
    uint256 private _setKycValidityReqTimestamp;

    constructor(
IPoolManager _poolManager,
address _kycValidity
    ) BaseHook(_poolManager) {
kycValidity = IKycValidity(_kycValidity);
    }


    modifier onlyPermitKYC() {
require(
    kycValidity.hasValidToken(tx.origin),
    "Swaps available for valid KYC token holders"
);
_;
    }

    /// Sorta timelock
    function setKycValidity(address _kycValidity) external onlyOwner {
if (
    block.timestamp - _setKycValidityReqTimestamp >= 7 days &&
    _kycValidity == _preKycValidity
) {
    kycValidity = IKycValidity(_kycValidity);
} else {
    _preKycValidity = _kycValidity;
    _setKycValidityReqTimestamp = block.timestamp;
}
    }

    function getHookPermissions()
public
pure
override
returns (Hooks.Permissions memory)
    {
return
    Hooks.Permissions({
        beforeInitialize: false,
        afterInitialize: false,
        beforeAddLiquidity: false,
        beforeRemoveLiquidity: false,
        afterAddLiquidity: false,
        afterRemoveLiquidity: false,
        beforeSwap: true,
        afterSwap: false,
        beforeDonate: false,
        afterDonate: false,
        beforeSwapReturnDelta: false,
        afterSwapReturnDelta: false,
        afterAddLiquidityReturnDelta: false,
        afterRemoveLiquidityReturnDelta: false
    });
    }

    function beforeSwap(
address,
PoolKey calldata,
IPoolManager.SwapParams calldata,
bytes calldata
    ) external view override onlyByManager onlyPermitKYC returns (bytes4, BeforeSwapDelta, uint24) {
return (this.beforeSwap.selector, BeforeSwapDeltaLibrary.ZERO_DELTA, 0);
    }



In [397]:
a = "// SPDX-License-Identifier: UNLICENSED\npragma solidity >=0.8.19;\n\nimport {Ownable} from \"@openzeppelin/contracts/access/Ownable.sol\";\nimport {IPoolManager} from \"v4-core/src/interfaces/IPoolManager.sol\";\nimport {Hooks} from \"v4-core/src/libraries/Hooks.sol\";\nimport {BaseHook} from \"v4-periphery/BaseHook.sol\";\nimport {PoolKey} from \"v4-core/src/types/PoolKey.sol\";\nimport {BeforeSwapDelta, BeforeSwapDeltaLibrary} from \"v4-core/src/types/BeforeSwapDelta.sol\";\n\n/**\n * @title An interface for checking whether an address has a valid kycNFT token\n */\ninterface IKycValidity {\n    /// @dev Check whether a given address has a valid kycNFT token\n    /// @param _addr Address to check for tokens\n    /// @return valid Whether the address has a valid token\n    function hasValidToken(address _addr) external view returns (bool valid);\n}\n\n/**\n * Only KYC'ed people can trade on the V4 hook'ed pool.\n * Caveat: Relies on external oracle for info on an address's KYC status.\n */\ncontract KYCSwaps is BaseHook, Ownable {\n    IKycValidity public kycValidity;\n    address private _preKycValidity;\n    uint256 private _setKycValidityReqTimestamp;\n\n    constructor(\n        IPoolManager _poolManager,\n        address _kycValidity\n    ) BaseHook(_poolManager) {\n        kycValidity = IKycValidity(_kycValidity);\n    }\n\n\n    modifier onlyPermitKYC() {\n        require(\n            kycValidity.hasValidToken(tx.origin),\n            \"Swaps available for valid KYC token holders\"\n        );\n        _;\n    }\n\n    /// Sorta timelock\n    function setKycValidity(address _kycValidity) external onlyOwner {\n        if (\n            block.timestamp - _setKycValidityReqTimestamp >= 7 days &&\n            _kycValidity == _preKycValidity\n        ) {\n            kycValidity = IKycValidity(_kycValidity);\n        } else {\n            _preKycValidity = _kycValidity;\n            _setKycValidityReqTimestamp = block.timestamp;\n        }\n    }\n\n    function getHookPermissions()\n        public\n        pure\n        override\n        returns (Hooks.Permissions memory)\n    {\n        return\n            Hooks.Permissions({\n                beforeInitialize: false,\n                afterInitialize: false,\n                beforeAddLiquidity: false,\n                beforeRemoveLiquidity: false,\n                afterAddLiquidity: false,\n                afterRemoveLiquidity: false,\n                beforeSwap: true,\n                afterSwap: false,\n                beforeDonate: false,\n                afterDonate: false,\n                beforeSwapReturnDelta: false,\n                afterSwapReturnDelta: false,\n                afterAddLiquidityReturnDelta: false,\n                afterRemoveLiquidityReturnDelta: false\n            });\n    }\n\n    function beforeSwap(\n        address,\n        PoolKey calldata,\n        IPoolManager.SwapParams calldata,\n        bytes calldata\n    ) external view override onlyByManager onlyPermitKYC returns (bytes4, BeforeSwapDelta, uint24) {\n        return (this.beforeSwap.selector, BeforeSwapDeltaLibrary.ZERO_DELTA, 0);\n    }\n\n    \n            function validateHookAddress(BaseHook _this) internal pure override {\n            }\n        }"

In [399]:
print("// SPDX-License-Identifier: UNLICENSED\npragma solidity >=0.8.19;\n\nimport {Ownable} from \"@openzeppelin/contracts/access/Ownable.sol\";\nimport {IPoolManager} from \"v4-core/src/interfaces/IPoolManager.sol\";\nimport {Hooks} from \"v4-core/src/libraries/Hooks.sol\";\nimport {BaseHook} from \"v4-periphery/BaseHook.sol\";\nimport {PoolKey} from \"v4-core/src/types/PoolKey.sol\";\nimport {BeforeSwapDelta, BeforeSwapDeltaLibrary} from \"v4-core/src/types/BeforeSwapDelta.sol\";\n\n/**\n * @title An interface for checking whether an address has a valid kycNFT token\n */\ninterface IKycValidity {\n    /// @dev Check whether a given address has a valid kycNFT token\n    /// @param _addr Address to check for tokens\n    /// @return valid Whether the address has a valid token\n    function hasValidToken(address _addr) external view returns (bool valid);\n}\n\n/**\n * Only KYC'ed people can trade on the V4 hook'ed pool.\n * Caveat: Relies on external oracle for info on an address's KYC status.\n */\ncontract KYCSwaps is BaseHook, Ownable {\n    IKycValidity public kycValidity;\n    address private _preKycValidity;\n    uint256 private _setKycValidityReqTimestamp;\n\n    constructor(\n        IPoolManager _poolManager,\n        address _kycValidity\n    ) BaseHook(_poolManager) {\n        kycValidity = IKycValidity(_kycValidity);\n    }\n\n\n    modifier onlyPermitKYC() {\n        require(\n            kycValidity.hasValidToken(tx.origin),\n            \"Swaps available for valid KYC token holders\"\n        );\n        _;\n    }\n\n    /// Sorta timelock\n    function setKycValidity(address _kycValidity) external onlyOwner {\n        if (\n            block.timestamp - _setKycValidityReqTimestamp >= 7 days &&\n            _kycValidity == _preKycValidity\n        ) {\n            kycValidity = IKycValidity(_kycValidity);\n        } else {\n            _preKycValidity = _kycValidity;\n            _setKycValidityReqTimestamp = block.timestamp;\n        }\n    }\n\n    function getHookPermissions()\n        public\n        pure\n        override\n        returns (Hooks.Permissions memory)\n    {\n        return\n            Hooks.Permissions({\n                beforeInitialize: false,\n                afterInitialize: false,\n                beforeAddLiquidity: false,\n                beforeRemoveLiquidity: false,\n                afterAddLiquidity: false,\n                afterRemoveLiquidity: false,\n                beforeSwap: true,\n                afterSwap: false,\n                beforeDonate: false,\n                afterDonate: false,\n                beforeSwapReturnDelta: false,\n                afterSwapReturnDelta: false,\n                afterAddLiquidityReturnDelta: false,\n                afterRemoveLiquidityReturnDelta: false\n            });\n    }\n\n    function beforeSwap(\n        address,\n        PoolKey calldata,\n        IPoolManager.SwapParams calldata,\n        bytes calldata\n    ) external view override onlyByManager onlyPermitKYC returns (bytes4, BeforeSwapDelta, uint24) {\n        return (this.beforeSwap.selector, BeforeSwapDeltaLibrary.ZERO_DELTA, 0);\n    }\n\n    \n            function validateHookAddress(BaseHook _this) internal pure override {\n            }\n        }")

// SPDX-License-Identifier: UNLICENSED
pragma solidity >=0.8.19;

import {Ownable} from "@openzeppelin/contracts/access/Ownable.sol";
import {IPoolManager} from "v4-core/src/interfaces/IPoolManager.sol";
import {Hooks} from "v4-core/src/libraries/Hooks.sol";
import {BaseHook} from "v4-periphery/BaseHook.sol";
import {PoolKey} from "v4-core/src/types/PoolKey.sol";
import {BeforeSwapDelta, BeforeSwapDeltaLibrary} from "v4-core/src/types/BeforeSwapDelta.sol";

/**
 * @title An interface for checking whether an address has a valid kycNFT token
 */
interface IKycValidity {
    /// @dev Check whether a given address has a valid kycNFT token
    /// @param _addr Address to check for tokens
    /// @return valid Whether the address has a valid token
    function hasValidToken(address _addr) external view returns (bool valid);
}

/**
 * Only KYC'ed people can trade on the V4 hook'ed pool.
 * Caveat: Relies on external oracle for info on an address's KYC status.
 */
contract KYCSwaps is BaseHook

In [420]:
import re

def n_arguments_in_constructor(text):
    """
    Count the number of commas between the parentheses after the constructor in the given text.
    
    :param text: The input string containing the constructor
    :return: The number of commas in the constructor's parameter list, or 0 if the list is empty or only contains spaces
    """
    # Regular expression to match the constructor parameters
    pattern = r'constructor\s*\(([^)]*)\)'
    match = re.search(pattern, text)
    
    if match:
        # Extract the parameters string
        params = match.group(1).strip()
        # Check if the parameters string is empty or only contains spaces
        if not params:
            return 0
        # Count the commas in the parameters string
        comma_count = params.count(',')
        return comma_count+1
    else:
        return 0

# Example usage
example_text = """
contract KYCSwaps is BaseHook, Ownable {
    IKycValidity public kycValidity;
    address private _preKycValidity;
    uint256 private _setKycValidityReqTimestamp;

    constructor( ,) BaseHook(_poolManager) {
        kycValidity = IKycValidity(_kycValidity);
    }

    // Other functions...
}
"""

result = n_arguments_in_constructor(example_text)
print(result)

2
